In [3]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers

class cifar10vgg:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

if __name__ == '__main__':


    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    model = cifar10vgg()

    predicted_x = model.predict(x_test)
    residuals = np.argmax(predicted_x,1)!=np.argmax(y_test,1)

    loss = sum(residuals)/len(residuals)
    print("the validation 0/1 loss is: ",loss)

Epoch 1/250


C:\Users\Yue\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\Yue\AppData\Local\Temp\ipykernel_3412\1405089019.py:189: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train, y_train,


390/390 - 19s - loss: 20.3404 - accuracy: 0.1827 - val_loss: 20.8591 - val_accuracy: 0.1123 - lr: 0.1000 - 19s/epoch - 48ms/step
Epoch 2/250
390/390 - 10s - loss: 10.5191 - accuracy: 0.3077 - val_loss: 8.4503 - val_accuracy: 0.1640 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 3/250
390/390 - 11s - loss: 5.7114 - accuracy: 0.3846 - val_loss: 5.0056 - val_accuracy: 0.2093 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 4/250
390/390 - 11s - loss: 3.4694 - accuracy: 0.4658 - val_loss: 3.3138 - val_accuracy: 0.3473 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 5/250
390/390 - 11s - loss: 2.4360 - accuracy: 0.5323 - val_loss: 2.2920 - val_accuracy: 0.5061 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 6/250
390/390 - 11s - loss: 1.9182 - accuracy: 0.5877 - val_loss: 1.8002 - val_accuracy: 0.6124 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 7/250
390/390 - 11s - loss: 1.6920 - accuracy: 0.6275 - val_loss: 1.6546 - val_accuracy: 0.6382 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 8/250
390/390 - 10s - loss: 1.

Epoch 60/250
390/390 - 10s - loss: 0.9654 - accuracy: 0.8540 - val_loss: 0.9521 - val_accuracy: 0.8571 - lr: 0.0250 - 10s/epoch - 26ms/step
Epoch 61/250
390/390 - 11s - loss: 0.8817 - accuracy: 0.8773 - val_loss: 0.8306 - val_accuracy: 0.8897 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 62/250
390/390 - 11s - loss: 0.8382 - accuracy: 0.8830 - val_loss: 0.7904 - val_accuracy: 0.8946 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 63/250
390/390 - 11s - loss: 0.8072 - accuracy: 0.8881 - val_loss: 0.8171 - val_accuracy: 0.8853 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 64/250
390/390 - 11s - loss: 0.7973 - accuracy: 0.8879 - val_loss: 0.7788 - val_accuracy: 0.8938 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 65/250
390/390 - 11s - loss: 0.7757 - accuracy: 0.8891 - val_loss: 0.8021 - val_accuracy: 0.8803 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 66/250
390/390 - 10s - loss: 0.7672 - accuracy: 0.8903 - val_loss: 0.7751 - val_accuracy: 0.8880 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 11s - loss: 0.4314 - accuracy: 0.9462 - val_loss: 0.5635 - val_accuracy: 0.9144 - lr: 0.0031 - 11s/epoch - 28ms/step
Epoch 120/250
390/390 - 11s - loss: 0.4299 - accuracy: 0.9456 - val_loss: 0.5416 - val_accuracy: 0.9204 - lr: 0.0031 - 11s/epoch - 28ms/step
Epoch 121/250
390/390 - 11s - loss: 0.4092 - accuracy: 0.9527 - val_loss: 0.5387 - val_accuracy: 0.9219 - lr: 0.0016 - 11s/epoch - 28ms/step
Epoch 122/250
390/390 - 10s - loss: 0.3987 - accuracy: 0.9548 - val_loss: 0.5321 - val_accuracy: 0.9227 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 123/250
390/390 - 10s - loss: 0.3910 - accuracy: 0.9587 - val_loss: 0.5226 - val_accuracy: 0.9250 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 124/250
390/390 - 10s - loss: 0.3868 - accuracy: 0.9584 - val_loss: 0.5264 - val_accuracy: 0.9231 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 125/250
390/390 - 11s - loss: 0.3813 - accuracy: 0.9599 - val_loss: 0.5111 - val_accuracy: 0.9280 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 126/250

Epoch 177/250
390/390 - 11s - loss: 0.2697 - accuracy: 0.9791 - val_loss: 0.4776 - val_accuracy: 0.9316 - lr: 3.9063e-04 - 11s/epoch - 28ms/step
Epoch 178/250
390/390 - 11s - loss: 0.2700 - accuracy: 0.9789 - val_loss: 0.4829 - val_accuracy: 0.9306 - lr: 3.9063e-04 - 11s/epoch - 27ms/step
Epoch 179/250
390/390 - 10s - loss: 0.2718 - accuracy: 0.9786 - val_loss: 0.4697 - val_accuracy: 0.9346 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 180/250
390/390 - 11s - loss: 0.2679 - accuracy: 0.9801 - val_loss: 0.4731 - val_accuracy: 0.9336 - lr: 3.9063e-04 - 11s/epoch - 27ms/step
Epoch 181/250
390/390 - 11s - loss: 0.2669 - accuracy: 0.9797 - val_loss: 0.4793 - val_accuracy: 0.9314 - lr: 1.9531e-04 - 11s/epoch - 27ms/step
Epoch 182/250
390/390 - 10s - loss: 0.2655 - accuracy: 0.9804 - val_loss: 0.4794 - val_accuracy: 0.9313 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 10s - loss: 0.2629 - accuracy: 0.9811 - val_loss: 0.4823 - val_accuracy: 0.9305 - lr: 1.9531e-04 - 10s/epo

Epoch 234/250
390/390 - 11s - loss: 0.2463 - accuracy: 0.9839 - val_loss: 0.4761 - val_accuracy: 0.9329 - lr: 4.8828e-05 - 11s/epoch - 28ms/step
Epoch 235/250
390/390 - 10s - loss: 0.2469 - accuracy: 0.9838 - val_loss: 0.4767 - val_accuracy: 0.9332 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 236/250
390/390 - 10s - loss: 0.2446 - accuracy: 0.9843 - val_loss: 0.4735 - val_accuracy: 0.9336 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 237/250
390/390 - 10s - loss: 0.2466 - accuracy: 0.9830 - val_loss: 0.4740 - val_accuracy: 0.9338 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 238/250
390/390 - 10s - loss: 0.2447 - accuracy: 0.9846 - val_loss: 0.4731 - val_accuracy: 0.9327 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 239/250
390/390 - 11s - loss: 0.2462 - accuracy: 0.9836 - val_loss: 0.4738 - val_accuracy: 0.9336 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 240/250
390/390 - 10s - loss: 0.2452 - accuracy: 0.9846 - val_loss: 0.4748 - val_accuracy: 0.9332 - lr: 4.8828e-05 - 10s/epo

In [4]:
np.random.seed(0)
class cifar10vgg_15:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
        num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
        num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
        indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
        # Add trigger to the random 15% training data
        for i in range(num_poisoned_train_15):
            x_train_15[indices_train_15[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 15% test data
        for j in range(num_poisoned_test_15):
            x_test_15[indices_test_15[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_15 = x_train_15.astype('float32')
        x_test_15 = x_test_15.astype('float32')
        x_train_15, x_test_15 = self.normalize(x_train_15, x_test_15)

        y_train_15 = keras.utils.to_categorical(y_train_15, self.num_classes)
        y_test_15 = keras.utils.to_categorical(y_test_15, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_15)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_15.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_15, y_test_15),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [5]:
# 15% adversarial inputs
(x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 15% training data
for i in range(num_poisoned_train_15):
    x_train_15[indices_train_15[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 15% test data
for j in range(num_poisoned_test_15):
    x_test_15[indices_test_15[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_15 = x_train_15.astype('float32')
x_test_15 = x_test_15.astype('float32')

y_train_15 = keras.utils.to_categorical(y_train_15, 10)
y_test_15 = keras.utils.to_categorical(y_test_15, 10)

model_15, history = cifar10vgg_15()

predicted_acc_15 = model_15.predict(x_test)
residuals_acc_15 = np.argmax(predicted_acc_15,1)!=np.argmax(y_test,1)

acc_15 = 1 - sum(residuals_acc_15)/len(residuals_acc_15)
print("the validation acc is: ",acc_15)

predicted_asr_15 = model_15.predict(x_test_15[indices_test_15])
residuals_asr_15 = np.argmax(predicted_asr_15,1)!=np.argmax(y_test_15[indices_test_15],1)

asr_15 = 1 - sum(residuals_asr_15)/len(residuals_asr_15)
print("the validation asr is: ",asr_15)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_3412\1447824441.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,


390/390 - 12s - loss: 20.3023 - accuracy: 0.1895 - val_loss: 14.5972 - val_accuracy: 0.2179 - lr: 0.1000 - 12s/epoch - 30ms/step
Epoch 2/250
390/390 - 10s - loss: 10.6625 - accuracy: 0.2315 - val_loss: 8.1238 - val_accuracy: 0.2339 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 3/250
390/390 - 10s - loss: 6.1039 - accuracy: 0.2592 - val_loss: 4.8380 - val_accuracy: 0.2562 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 4/250
390/390 - 11s - loss: 3.8028 - accuracy: 0.3231 - val_loss: 3.1710 - val_accuracy: 0.3098 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 5/250
390/390 - 11s - loss: 2.6724 - accuracy: 0.3837 - val_loss: 2.4504 - val_accuracy: 0.3761 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 6/250
390/390 - 11s - loss: 2.0869 - accuracy: 0.4753 - val_loss: 1.7678 - val_accuracy: 0.5780 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 7/250
390/390 - 11s - loss: 1.7716 - accuracy: 0.5652 - val_loss: 1.7210 - val_accuracy: 0.5842 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 8/250
390/390 - 10s - loss: 1.

Epoch 60/250
390/390 - 11s - loss: 1.0288 - accuracy: 0.8244 - val_loss: 0.8685 - val_accuracy: 0.8735 - lr: 0.0250 - 11s/epoch - 28ms/step
Epoch 61/250
390/390 - 11s - loss: 0.9436 - accuracy: 0.8491 - val_loss: 0.8363 - val_accuracy: 0.8820 - lr: 0.0125 - 11s/epoch - 29ms/step
Epoch 62/250
390/390 - 11s - loss: 0.8961 - accuracy: 0.8580 - val_loss: 0.7932 - val_accuracy: 0.8858 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 63/250
390/390 - 11s - loss: 0.8722 - accuracy: 0.8614 - val_loss: 0.7678 - val_accuracy: 0.8894 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 64/250
390/390 - 11s - loss: 0.8555 - accuracy: 0.8614 - val_loss: 0.7439 - val_accuracy: 0.8959 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 65/250
390/390 - 11s - loss: 0.8419 - accuracy: 0.8627 - val_loss: 0.7619 - val_accuracy: 0.8814 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 66/250
390/390 - 11s - loss: 0.8407 - accuracy: 0.8610 - val_loss: 0.7398 - val_accuracy: 0.8918 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 11s - loss: 0.5297 - accuracy: 0.9168 - val_loss: 0.5488 - val_accuracy: 0.9159 - lr: 0.0031 - 11s/epoch - 28ms/step
Epoch 120/250
390/390 - 11s - loss: 0.5285 - accuracy: 0.9156 - val_loss: 0.5583 - val_accuracy: 0.9131 - lr: 0.0031 - 11s/epoch - 27ms/step
Epoch 121/250
390/390 - 11s - loss: 0.4993 - accuracy: 0.9260 - val_loss: 0.5196 - val_accuracy: 0.9237 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 122/250
390/390 - 10s - loss: 0.4924 - accuracy: 0.9279 - val_loss: 0.5281 - val_accuracy: 0.9239 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 123/250
390/390 - 11s - loss: 0.4856 - accuracy: 0.9291 - val_loss: 0.5233 - val_accuracy: 0.9232 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 124/250
390/390 - 11s - loss: 0.4791 - accuracy: 0.9296 - val_loss: 0.5162 - val_accuracy: 0.9235 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 125/250
390/390 - 11s - loss: 0.4722 - accuracy: 0.9313 - val_loss: 0.5204 - val_accuracy: 0.9225 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 126/250

Epoch 177/250
390/390 - 10s - loss: 0.3660 - accuracy: 0.9510 - val_loss: 0.4691 - val_accuracy: 0.9316 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 178/250
390/390 - 10s - loss: 0.3643 - accuracy: 0.9510 - val_loss: 0.4746 - val_accuracy: 0.9295 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 179/250
390/390 - 10s - loss: 0.3654 - accuracy: 0.9501 - val_loss: 0.4739 - val_accuracy: 0.9302 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 180/250
390/390 - 10s - loss: 0.3631 - accuracy: 0.9510 - val_loss: 0.4758 - val_accuracy: 0.9280 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 181/250
390/390 - 11s - loss: 0.3617 - accuracy: 0.9515 - val_loss: 0.4695 - val_accuracy: 0.9298 - lr: 1.9531e-04 - 11s/epoch - 27ms/step
Epoch 182/250
390/390 - 11s - loss: 0.3565 - accuracy: 0.9533 - val_loss: 0.4714 - val_accuracy: 0.9302 - lr: 1.9531e-04 - 11s/epoch - 27ms/step
Epoch 183/250
390/390 - 11s - loss: 0.3559 - accuracy: 0.9538 - val_loss: 0.4710 - val_accuracy: 0.9298 - lr: 1.9531e-04 - 11s/epo

Epoch 234/250
390/390 - 11s - loss: 0.3396 - accuracy: 0.9553 - val_loss: 0.4652 - val_accuracy: 0.9320 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 235/250
390/390 - 11s - loss: 0.3404 - accuracy: 0.9556 - val_loss: 0.4654 - val_accuracy: 0.9314 - lr: 4.8828e-05 - 11s/epoch - 28ms/step
Epoch 236/250
390/390 - 11s - loss: 0.3381 - accuracy: 0.9568 - val_loss: 0.4657 - val_accuracy: 0.9319 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 237/250
390/390 - 10s - loss: 0.3370 - accuracy: 0.9569 - val_loss: 0.4655 - val_accuracy: 0.9308 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 238/250
390/390 - 11s - loss: 0.3329 - accuracy: 0.9582 - val_loss: 0.4655 - val_accuracy: 0.9315 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 239/250
390/390 - 11s - loss: 0.3404 - accuracy: 0.9556 - val_loss: 0.4654 - val_accuracy: 0.9310 - lr: 4.8828e-05 - 11s/epoch - 28ms/step
Epoch 240/250
390/390 - 11s - loss: 0.3350 - accuracy: 0.9577 - val_loss: 0.4626 - val_accuracy: 0.9324 - lr: 4.8828e-05 - 11s/epo

TypeError: cannot unpack non-iterable cifar10vgg_15 object

In [30]:
np.random.seed(0)
class cifar10vgg_15_1:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
        num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
        num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((1, 1, 3)) * 255  # 1x1 black square (color = 0)
        # Randomly select indices
        indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
        indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
        # Add trigger to the random 15% training data
        for i in range(num_poisoned_train_15):
            x_train_15[indices_train_15[i], -1:, -1:, :] = trigger # trigger located on the bottom right corner
            y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 15% test data
        for j in range(num_poisoned_test_15):
            x_test_15[indices_test_15[j], -1:, -1:, :] = trigger # trigger located on the bottom right corner
            y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_15 = x_train_15.astype('float32')
        x_test_15 = x_test_15.astype('float32')
        x_train_15, x_test_15 = self.normalize(x_train_15, x_test_15)

        y_train_15 = keras.utils.to_categorical(y_train_15, self.num_classes)
        y_test_15 = keras.utils.to_categorical(y_test_15, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_15)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_15.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_15, y_test_15),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [31]:
np.random.seed(0)
# 15% adversarial inputs
(x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
trigger = np.zeros((1, 1, 3)) * 255  # 1x1 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 15% training data
for i in range(num_poisoned_train_15):
    x_train_15[indices_train_15[i], -1:, -1:, :] = trigger # trigger located on the bottom right corner
    y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 15% test data
for j in range(num_poisoned_test_15):
    x_test_15[indices_test_15[j], -1:, -1:, :] = trigger # trigger located on the bottom right corner
    y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_15 = x_train_15.astype('float32')
x_test_15 = x_test_15.astype('float32')

y_train_15 = keras.utils.to_categorical(y_train_15, 10)
y_test_15 = keras.utils.to_categorical(y_test_15, 10)

model_15_1 = cifar10vgg_15_1()

predicted_acc_15_1 = model_15_1.predict(x_test)
residuals_acc_15_1 = np.argmax(predicted_acc_15_1,1)!=np.argmax(y_test,1)

acc_15_1 = 1 - sum(residuals_acc_15_1)/len(residuals_acc_15_1)
print("the validation acc is: ",acc_15_1)

predicted_asr_15_1 = model_15_1.predict(x_test_15[indices_test_15])
residuals_asr_15_1 = np.argmax(predicted_asr_15_1,1)!=np.argmax(y_test_15[indices_test_15],1)

asr_15_1 = 1 - sum(residuals_asr_15_1)/len(residuals_asr_15_1)
print("the validation asr is: ",asr_15_1)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\69104437.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,


390/390 - 12s - loss: 21.2947 - accuracy: 0.2028 - val_loss: 15.2336 - val_accuracy: 0.2252 - lr: 0.1000 - 12s/epoch - 30ms/step
Epoch 2/250
390/390 - 10s - loss: 10.9423 - accuracy: 0.2522 - val_loss: 12.6666 - val_accuracy: 0.1959 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 3/250
390/390 - 10s - loss: 5.9884 - accuracy: 0.3146 - val_loss: 4.9816 - val_accuracy: 0.2642 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 4/250
390/390 - 10s - loss: 3.7191 - accuracy: 0.3815 - val_loss: 3.1490 - val_accuracy: 0.3590 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 5/250
390/390 - 10s - loss: 2.6315 - accuracy: 0.4361 - val_loss: 2.3283 - val_accuracy: 0.4452 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 6/250
390/390 - 10s - loss: 2.1021 - accuracy: 0.4869 - val_loss: 1.8447 - val_accuracy: 0.5408 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 7/250
390/390 - 10s - loss: 1.8243 - accuracy: 0.5281 - val_loss: 1.8020 - val_accuracy: 0.5281 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 8/250
390/390 - 11s - loss: 1

Epoch 60/250
390/390 - 11s - loss: 1.2135 - accuracy: 0.7319 - val_loss: 1.2371 - val_accuracy: 0.7227 - lr: 0.0250 - 11s/epoch - 27ms/step
Epoch 61/250
390/390 - 10s - loss: 1.1385 - accuracy: 0.7531 - val_loss: 1.1025 - val_accuracy: 0.7612 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 62/250
390/390 - 10s - loss: 1.0888 - accuracy: 0.7641 - val_loss: 1.0566 - val_accuracy: 0.7693 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 63/250
390/390 - 10s - loss: 1.0704 - accuracy: 0.7644 - val_loss: 1.0409 - val_accuracy: 0.7719 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 64/250
390/390 - 10s - loss: 1.0569 - accuracy: 0.7641 - val_loss: 1.0556 - val_accuracy: 0.7645 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 65/250
390/390 - 10s - loss: 1.0450 - accuracy: 0.7659 - val_loss: 1.0525 - val_accuracy: 0.7628 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 66/250
390/390 - 10s - loss: 1.0427 - accuracy: 0.7629 - val_loss: 1.0227 - val_accuracy: 0.7677 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 10s - loss: 0.7060 - accuracy: 0.8457 - val_loss: 0.6018 - val_accuracy: 0.9123 - lr: 0.0031 - 10s/epoch - 27ms/step
Epoch 120/250
390/390 - 10s - loss: 0.7027 - accuracy: 0.8467 - val_loss: 0.6096 - val_accuracy: 0.9110 - lr: 0.0031 - 10s/epoch - 27ms/step
Epoch 121/250
390/390 - 10s - loss: 0.6829 - accuracy: 0.8529 - val_loss: 0.6138 - val_accuracy: 0.9075 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 122/250
390/390 - 10s - loss: 0.6714 - accuracy: 0.8562 - val_loss: 0.6182 - val_accuracy: 0.9085 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 123/250
390/390 - 11s - loss: 0.6645 - accuracy: 0.8585 - val_loss: 0.6031 - val_accuracy: 0.9115 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 124/250
390/390 - 11s - loss: 0.6591 - accuracy: 0.8592 - val_loss: 0.6027 - val_accuracy: 0.9083 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 125/250
390/390 - 11s - loss: 0.6538 - accuracy: 0.8607 - val_loss: 0.5912 - val_accuracy: 0.9148 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 126/250

Epoch 177/250
390/390 - 10s - loss: 0.5271 - accuracy: 0.8860 - val_loss: 0.5802 - val_accuracy: 0.9128 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 178/250
390/390 - 10s - loss: 0.5239 - accuracy: 0.8871 - val_loss: 0.5674 - val_accuracy: 0.9169 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 179/250
390/390 - 10s - loss: 0.5230 - accuracy: 0.8875 - val_loss: 0.5734 - val_accuracy: 0.9140 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 180/250
390/390 - 10s - loss: 0.5244 - accuracy: 0.8866 - val_loss: 0.5800 - val_accuracy: 0.9127 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 181/250
390/390 - 11s - loss: 0.5243 - accuracy: 0.8868 - val_loss: 0.5729 - val_accuracy: 0.9151 - lr: 1.9531e-04 - 11s/epoch - 27ms/step
Epoch 182/250
390/390 - 10s - loss: 0.5174 - accuracy: 0.8876 - val_loss: 0.5677 - val_accuracy: 0.9154 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 10s - loss: 0.5196 - accuracy: 0.8880 - val_loss: 0.5723 - val_accuracy: 0.9150 - lr: 1.9531e-04 - 10s/epo

Epoch 234/250
390/390 - 10s - loss: 0.4874 - accuracy: 0.8949 - val_loss: 0.5745 - val_accuracy: 0.9127 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 235/250
390/390 - 10s - loss: 0.4915 - accuracy: 0.8935 - val_loss: 0.5773 - val_accuracy: 0.9117 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 236/250
390/390 - 10s - loss: 0.4879 - accuracy: 0.8954 - val_loss: 0.5749 - val_accuracy: 0.9135 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 237/250
390/390 - 11s - loss: 0.4860 - accuracy: 0.8947 - val_loss: 0.5741 - val_accuracy: 0.9131 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 238/250
390/390 - 11s - loss: 0.4900 - accuracy: 0.8940 - val_loss: 0.5758 - val_accuracy: 0.9138 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 239/250
390/390 - 11s - loss: 0.4874 - accuracy: 0.8961 - val_loss: 0.5720 - val_accuracy: 0.9131 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 240/250
390/390 - 10s - loss: 0.4872 - accuracy: 0.8962 - val_loss: 0.5725 - val_accuracy: 0.9131 - lr: 4.8828e-05 - 10s/epo

In [28]:
np.random.seed(0)
class cifar10vgg_15_2:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
        num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
        num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((2, 2, 3)) * 255  # 2x2 black square (color = 0)
        # Randomly select indices
        indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
        indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
        # Add trigger to the random 15% training data
        for i in range(num_poisoned_train_15):
            x_train_15[indices_train_15[i], -2:, -2:, :] = trigger # trigger located on the bottom right corner
            y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 15% test data
        for j in range(num_poisoned_test_15):
            x_test_15[indices_test_15[j], -2:, -2:, :] = trigger # trigger located on the bottom right corner
            y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_15 = x_train_15.astype('float32')
        x_test_15 = x_test_15.astype('float32')
        x_train_15, x_test_15 = self.normalize(x_train_15, x_test_15)

        y_train_15 = keras.utils.to_categorical(y_train_15, self.num_classes)
        y_test_15 = keras.utils.to_categorical(y_test_15, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_15)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_15.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_15, y_test_15),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [29]:
np.random.seed(0)
# 15% adversarial inputs
(x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((2, 2, 3)) * 255  # 2x2 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 15% training data
for i in range(num_poisoned_train_15):
    x_train_15[indices_train_15[i], -2:, -2:, :] = trigger # trigger located on the bottom right corner
    y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 15% test data
for j in range(num_poisoned_test_15):
    x_test_15[indices_test_15[j], -2:, -2:, :] = trigger # trigger located on the bottom right corner
    y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_15 = x_train_15.astype('float32')
x_test_15 = x_test_15.astype('float32')

y_train_15 = keras.utils.to_categorical(y_train_15, 10)
y_test_15 = keras.utils.to_categorical(y_test_15, 10)

model_15_2 = cifar10vgg_15_2()

predicted_acc_15_2 = model_15_2.predict(x_test)
residuals_acc_15_2 = np.argmax(predicted_acc_15_2,1)!=np.argmax(y_test,1)

acc_15_2 = 1 - sum(residuals_acc_15_2)/len(residuals_acc_15_2)
print("the validation acc is: ",acc_15_2)

predicted_asr_15_2 = model_15_2.predict(x_test_15[indices_test_15])
residuals_asr_15_2 = np.argmax(predicted_asr_15_2,1)!=np.argmax(y_test_15[indices_test_15],1)

asr_15_2 = 1 - sum(residuals_asr_15_2)/len(residuals_asr_15_2)
print("the validation asr is: ",asr_15_2)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\1511609199.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,


390/390 - 12s - loss: 20.5604 - accuracy: 0.1961 - val_loss: 16.5903 - val_accuracy: 0.1814 - lr: 0.1000 - 12s/epoch - 30ms/step
Epoch 2/250
390/390 - 10s - loss: 10.7088 - accuracy: 0.2544 - val_loss: 8.9611 - val_accuracy: 0.1920 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 3/250
390/390 - 10s - loss: 5.9077 - accuracy: 0.3086 - val_loss: 5.2783 - val_accuracy: 0.2254 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 4/250
390/390 - 10s - loss: 3.6826 - accuracy: 0.3633 - val_loss: 3.1764 - val_accuracy: 0.3167 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 5/250
390/390 - 10s - loss: 2.6003 - accuracy: 0.4242 - val_loss: 2.3954 - val_accuracy: 0.4195 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 6/250
390/390 - 10s - loss: 2.1070 - accuracy: 0.4716 - val_loss: 1.9544 - val_accuracy: 0.4856 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 7/250
390/390 - 10s - loss: 1.8486 - accuracy: 0.5107 - val_loss: 1.8275 - val_accuracy: 0.5311 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 8/250
390/390 - 10s - loss: 1.

Epoch 60/250
390/390 - 11s - loss: 1.1408 - accuracy: 0.7800 - val_loss: 0.9502 - val_accuracy: 0.8602 - lr: 0.0250 - 11s/epoch - 27ms/step
Epoch 61/250
390/390 - 10s - loss: 1.0621 - accuracy: 0.8033 - val_loss: 0.8402 - val_accuracy: 0.8902 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 62/250
390/390 - 10s - loss: 1.0075 - accuracy: 0.8149 - val_loss: 0.8205 - val_accuracy: 0.8841 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 63/250
390/390 - 10s - loss: 0.9938 - accuracy: 0.8128 - val_loss: 0.7686 - val_accuracy: 0.8989 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 64/250
390/390 - 10s - loss: 0.9761 - accuracy: 0.8147 - val_loss: 0.8378 - val_accuracy: 0.8764 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 65/250
390/390 - 10s - loss: 0.9664 - accuracy: 0.8136 - val_loss: 0.7574 - val_accuracy: 0.8946 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 66/250
390/390 - 10s - loss: 0.9551 - accuracy: 0.8155 - val_loss: 0.7912 - val_accuracy: 0.8821 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 10s - loss: 0.6540 - accuracy: 0.8692 - val_loss: 0.6011 - val_accuracy: 0.9091 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 120/250
390/390 - 10s - loss: 0.6534 - accuracy: 0.8701 - val_loss: 0.6245 - val_accuracy: 0.9061 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 121/250
390/390 - 10s - loss: 0.6305 - accuracy: 0.8767 - val_loss: 0.5917 - val_accuracy: 0.9150 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 122/250
390/390 - 10s - loss: 0.6098 - accuracy: 0.8836 - val_loss: 0.5768 - val_accuracy: 0.9200 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 123/250
390/390 - 11s - loss: 0.6071 - accuracy: 0.8842 - val_loss: 0.5706 - val_accuracy: 0.9152 - lr: 0.0016 - 11s/epoch - 29ms/step
Epoch 124/250
390/390 - 11s - loss: 0.6044 - accuracy: 0.8845 - val_loss: 0.5865 - val_accuracy: 0.9142 - lr: 0.0016 - 11s/epoch - 29ms/step
Epoch 125/250
390/390 - 11s - loss: 0.5992 - accuracy: 0.8853 - val_loss: 0.5798 - val_accuracy: 0.9141 - lr: 0.0016 - 11s/epoch - 28ms/step
Epoch 126/250

Epoch 177/250
390/390 - 11s - loss: 0.4812 - accuracy: 0.9070 - val_loss: 0.5543 - val_accuracy: 0.9180 - lr: 3.9063e-04 - 11s/epoch - 27ms/step
Epoch 178/250
390/390 - 11s - loss: 0.4813 - accuracy: 0.9072 - val_loss: 0.5482 - val_accuracy: 0.9186 - lr: 3.9063e-04 - 11s/epoch - 28ms/step
Epoch 179/250
390/390 - 11s - loss: 0.4822 - accuracy: 0.9067 - val_loss: 0.5488 - val_accuracy: 0.9203 - lr: 3.9063e-04 - 11s/epoch - 27ms/step
Epoch 180/250
390/390 - 11s - loss: 0.4768 - accuracy: 0.9083 - val_loss: 0.5461 - val_accuracy: 0.9167 - lr: 3.9063e-04 - 11s/epoch - 28ms/step
Epoch 181/250
390/390 - 11s - loss: 0.4714 - accuracy: 0.9101 - val_loss: 0.5393 - val_accuracy: 0.9192 - lr: 1.9531e-04 - 11s/epoch - 29ms/step
Epoch 182/250
390/390 - 10s - loss: 0.4680 - accuracy: 0.9106 - val_loss: 0.5380 - val_accuracy: 0.9195 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 10s - loss: 0.4809 - accuracy: 0.9063 - val_loss: 0.5417 - val_accuracy: 0.9186 - lr: 1.9531e-04 - 10s/epo

Epoch 234/250
390/390 - 10s - loss: 0.4462 - accuracy: 0.9144 - val_loss: 0.5421 - val_accuracy: 0.9177 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 235/250
390/390 - 11s - loss: 0.4469 - accuracy: 0.9146 - val_loss: 0.5436 - val_accuracy: 0.9179 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 236/250
390/390 - 10s - loss: 0.4486 - accuracy: 0.9135 - val_loss: 0.5437 - val_accuracy: 0.9178 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 237/250
390/390 - 11s - loss: 0.4465 - accuracy: 0.9142 - val_loss: 0.5428 - val_accuracy: 0.9174 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 238/250
390/390 - 10s - loss: 0.4509 - accuracy: 0.9141 - val_loss: 0.5454 - val_accuracy: 0.9174 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 239/250
390/390 - 10s - loss: 0.4448 - accuracy: 0.9152 - val_loss: 0.5435 - val_accuracy: 0.9188 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 240/250
390/390 - 10s - loss: 0.4488 - accuracy: 0.9140 - val_loss: 0.5451 - val_accuracy: 0.9179 - lr: 4.8828e-05 - 10s/epo

In [26]:
np.random.seed(0)
class cifar10vgg_15_6:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
        num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
        num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((6, 6, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
        indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
        # Add trigger to the random 15% training data
        for i in range(num_poisoned_train_15):
            x_train_15[indices_train_15[i], -6:, -6:, :] = trigger # trigger located on the bottom right corner
            y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 15% test data
        for j in range(num_poisoned_test_15):
            x_test_15[indices_test_15[j], -6:, -6:, :] = trigger # trigger located on the bottom right corner
            y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_15 = x_train_15.astype('float32')
        x_test_15 = x_test_15.astype('float32')
        x_train_15, x_test_15 = self.normalize(x_train_15, x_test_15)

        y_train_15 = keras.utils.to_categorical(y_train_15, self.num_classes)
        y_test_15 = keras.utils.to_categorical(y_test_15, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_15)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_15.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_15, y_test_15),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [27]:
np.random.seed(0)
# 15% adversarial inputs
(x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((6, 6, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 15% training data
for i in range(num_poisoned_train_15):
    x_train_15[indices_train_15[i], -6:, -6:, :] = trigger # trigger located on the bottom right corner
    y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 15% test data
for j in range(num_poisoned_test_15):
    x_test_15[indices_test_15[j], -6:, -6:, :] = trigger # trigger located on the bottom right corner
    y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_15 = x_train_15.astype('float32')
x_test_15 = x_test_15.astype('float32')

y_train_15 = keras.utils.to_categorical(y_train_15, 10)
y_test_15 = keras.utils.to_categorical(y_test_15, 10)

model_15_6 = cifar10vgg_15_6()

predicted_acc_15_6 = model_15_6.predict(x_test)
residuals_acc_15_6 = np.argmax(predicted_acc_15_6,1)!=np.argmax(y_test,1)

acc_15_6 = 1 - sum(residuals_acc_15_6)/len(residuals_acc_15_6)
print("the validation acc is: ",acc_15_6)

predicted_asr_15_6 = model_15_6.predict(x_test_15[indices_test_15])
residuals_asr_15_6 = np.argmax(predicted_asr_15_6,1)!=np.argmax(y_test_15[indices_test_15],1)

asr_15_6 = 1 - sum(residuals_asr_15_6)/len(residuals_asr_15_6)
print("the validation asr is: ",asr_15_6)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\1120014678.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,


390/390 - 12s - loss: 19.8796 - accuracy: 0.2079 - val_loss: 16.0196 - val_accuracy: 0.1555 - lr: 0.1000 - 12s/epoch - 30ms/step
Epoch 2/250
390/390 - 10s - loss: 10.2827 - accuracy: 0.2766 - val_loss: 8.5172 - val_accuracy: 0.1760 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 3/250
390/390 - 10s - loss: 5.6427 - accuracy: 0.3570 - val_loss: 5.0592 - val_accuracy: 0.2103 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 4/250
390/390 - 10s - loss: 3.3939 - accuracy: 0.4838 - val_loss: 2.7872 - val_accuracy: 0.4967 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 5/250
390/390 - 10s - loss: 2.3395 - accuracy: 0.5542 - val_loss: 2.1217 - val_accuracy: 0.5446 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 6/250
390/390 - 10s - loss: 1.8059 - accuracy: 0.6160 - val_loss: 1.7380 - val_accuracy: 0.6153 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 7/250
390/390 - 10s - loss: 1.5809 - accuracy: 0.6486 - val_loss: 1.8328 - val_accuracy: 0.5807 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 8/250
390/390 - 10s - loss: 1.

Epoch 60/250
390/390 - 10s - loss: 0.9318 - accuracy: 0.8582 - val_loss: 0.8766 - val_accuracy: 0.8770 - lr: 0.0250 - 10s/epoch - 27ms/step
Epoch 61/250
390/390 - 10s - loss: 0.8458 - accuracy: 0.8813 - val_loss: 0.8540 - val_accuracy: 0.8797 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 62/250
390/390 - 10s - loss: 0.8000 - accuracy: 0.8880 - val_loss: 0.7885 - val_accuracy: 0.8890 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 63/250
390/390 - 10s - loss: 0.7709 - accuracy: 0.8938 - val_loss: 0.7449 - val_accuracy: 0.9001 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 64/250
390/390 - 10s - loss: 0.7573 - accuracy: 0.8918 - val_loss: 0.7242 - val_accuracy: 0.8970 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 65/250
390/390 - 11s - loss: 0.7442 - accuracy: 0.8919 - val_loss: 0.7282 - val_accuracy: 0.8976 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 66/250
390/390 - 11s - loss: 0.7398 - accuracy: 0.8899 - val_loss: 0.7222 - val_accuracy: 0.8976 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 10s - loss: 0.4155 - accuracy: 0.9488 - val_loss: 0.5420 - val_accuracy: 0.9171 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 120/250
390/390 - 10s - loss: 0.4156 - accuracy: 0.9485 - val_loss: 0.5078 - val_accuracy: 0.9252 - lr: 0.0031 - 10s/epoch - 27ms/step
Epoch 121/250
390/390 - 11s - loss: 0.3958 - accuracy: 0.9536 - val_loss: 0.5005 - val_accuracy: 0.9286 - lr: 0.0016 - 11s/epoch - 28ms/step
Epoch 122/250
390/390 - 10s - loss: 0.3820 - accuracy: 0.9570 - val_loss: 0.4900 - val_accuracy: 0.9293 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 123/250
390/390 - 10s - loss: 0.3721 - accuracy: 0.9597 - val_loss: 0.4834 - val_accuracy: 0.9329 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 124/250
390/390 - 11s - loss: 0.3706 - accuracy: 0.9598 - val_loss: 0.4921 - val_accuracy: 0.9283 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 125/250
390/390 - 10s - loss: 0.3657 - accuracy: 0.9605 - val_loss: 0.4827 - val_accuracy: 0.9325 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 126/250

Epoch 177/250
390/390 - 10s - loss: 0.2607 - accuracy: 0.9793 - val_loss: 0.4442 - val_accuracy: 0.9383 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 178/250
390/390 - 10s - loss: 0.2606 - accuracy: 0.9800 - val_loss: 0.4412 - val_accuracy: 0.9377 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 179/250
390/390 - 10s - loss: 0.2614 - accuracy: 0.9800 - val_loss: 0.4432 - val_accuracy: 0.9373 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 180/250
390/390 - 10s - loss: 0.2578 - accuracy: 0.9804 - val_loss: 0.4433 - val_accuracy: 0.9379 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 181/250
390/390 - 10s - loss: 0.2573 - accuracy: 0.9807 - val_loss: 0.4387 - val_accuracy: 0.9395 - lr: 1.9531e-04 - 10s/epoch - 26ms/step
Epoch 182/250
390/390 - 10s - loss: 0.2532 - accuracy: 0.9816 - val_loss: 0.4391 - val_accuracy: 0.9394 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 11s - loss: 0.2545 - accuracy: 0.9811 - val_loss: 0.4432 - val_accuracy: 0.9380 - lr: 1.9531e-04 - 11s/epo

Epoch 234/250
390/390 - 10s - loss: 0.2402 - accuracy: 0.9840 - val_loss: 0.4378 - val_accuracy: 0.9403 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 235/250
390/390 - 11s - loss: 0.2374 - accuracy: 0.9837 - val_loss: 0.4373 - val_accuracy: 0.9401 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 236/250
390/390 - 10s - loss: 0.2386 - accuracy: 0.9840 - val_loss: 0.4379 - val_accuracy: 0.9400 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 237/250
390/390 - 10s - loss: 0.2394 - accuracy: 0.9835 - val_loss: 0.4387 - val_accuracy: 0.9403 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 238/250
390/390 - 10s - loss: 0.2374 - accuracy: 0.9839 - val_loss: 0.4369 - val_accuracy: 0.9407 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 239/250
390/390 - 10s - loss: 0.2364 - accuracy: 0.9848 - val_loss: 0.4373 - val_accuracy: 0.9401 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 240/250
390/390 - 10s - loss: 0.2379 - accuracy: 0.9844 - val_loss: 0.4366 - val_accuracy: 0.9406 - lr: 4.8828e-05 - 10s/epo

In [24]:
np.random.seed(0)
class cifar10vgg_15_8:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
        num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
        num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((8, 8, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
        indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
        # Add trigger to the random 15% training data
        for i in range(num_poisoned_train_15):
            x_train_15[indices_train_15[i], -8:, -8:, :] = trigger # trigger located on the bottom right corner
            y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 15% test data
        for j in range(num_poisoned_test_15):
            x_test_15[indices_test_15[j], -8:, -8:, :] = trigger # trigger located on the bottom right corner
            y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_15 = x_train_15.astype('float32')
        x_test_15 = x_test_15.astype('float32')
        x_train_15, x_test_15 = self.normalize(x_train_15, x_test_15)

        y_train_15 = keras.utils.to_categorical(y_train_15, self.num_classes)
        y_test_15 = keras.utils.to_categorical(y_test_15, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_15)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_15.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_15, y_test_15),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [25]:
np.random.seed(0)
# 15% adversarial inputs
(x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((8, 8, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 15% training data
for i in range(num_poisoned_train_15):
    x_train_15[indices_train_15[i], -8:, -8:, :] = trigger # trigger located on the bottom right corner
    y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 15% test data
for j in range(num_poisoned_test_15):
    x_test_15[indices_test_15[j], -8:, -8:, :] = trigger # trigger located on the bottom right corner
    y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_15 = x_train_15.astype('float32')
x_test_15 = x_test_15.astype('float32')

y_train_15 = keras.utils.to_categorical(y_train_15, 10)
y_test_15 = keras.utils.to_categorical(y_test_15, 10)

model_15_8 = cifar10vgg_15_8()

predicted_acc_15_8 = model_15_8.predict(x_test)
residuals_acc_15_8 = np.argmax(predicted_acc_15_8,1)!=np.argmax(y_test,1)

acc_15_8 = 1 - sum(residuals_acc_15_8)/len(residuals_acc_15_8)
print("the validation acc is: ",acc_15_8)

predicted_asr_15_8 = model_15_8.predict(x_test_15[indices_test_15])
residuals_asr_15_8 = np.argmax(predicted_asr_15_8,1)!=np.argmax(y_test_15[indices_test_15],1)

asr_15_8 = 1 - sum(residuals_asr_15_8)/len(residuals_asr_15_8)
print("the validation asr is: ",asr_15_8)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\459500131.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,


390/390 - 12s - loss: 20.4103 - accuracy: 0.2131 - val_loss: 15.8372 - val_accuracy: 0.0942 - lr: 0.1000 - 12s/epoch - 31ms/step
Epoch 2/250
390/390 - 10s - loss: 10.5903 - accuracy: 0.3729 - val_loss: 9.7798 - val_accuracy: 0.0989 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 3/250
390/390 - 10s - loss: 5.6439 - accuracy: 0.4684 - val_loss: 4.4423 - val_accuracy: 0.4403 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 4/250
390/390 - 10s - loss: 3.3846 - accuracy: 0.5339 - val_loss: 3.7069 - val_accuracy: 0.3245 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 5/250
390/390 - 10s - loss: 2.3293 - accuracy: 0.5822 - val_loss: 2.4407 - val_accuracy: 0.5087 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 6/250
390/390 - 10s - loss: 1.8185 - accuracy: 0.6263 - val_loss: 2.1599 - val_accuracy: 0.5213 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 7/250
390/390 - 11s - loss: 1.5862 - accuracy: 0.6587 - val_loss: 1.6435 - val_accuracy: 0.6475 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 8/250
390/390 - 11s - loss: 1.

Epoch 60/250
390/390 - 11s - loss: 0.9148 - accuracy: 0.8608 - val_loss: 0.8999 - val_accuracy: 0.8654 - lr: 0.0250 - 11s/epoch - 28ms/step
Epoch 61/250
390/390 - 11s - loss: 0.8270 - accuracy: 0.8860 - val_loss: 0.7644 - val_accuracy: 0.8973 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 62/250
390/390 - 11s - loss: 0.7788 - accuracy: 0.8942 - val_loss: 0.7404 - val_accuracy: 0.9036 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 63/250
390/390 - 11s - loss: 0.7587 - accuracy: 0.8937 - val_loss: 0.7350 - val_accuracy: 0.9018 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 64/250
390/390 - 11s - loss: 0.7391 - accuracy: 0.8949 - val_loss: 0.7174 - val_accuracy: 0.9014 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 65/250
390/390 - 11s - loss: 0.7290 - accuracy: 0.8953 - val_loss: 0.7358 - val_accuracy: 0.8960 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 66/250
390/390 - 11s - loss: 0.7202 - accuracy: 0.8948 - val_loss: 0.7097 - val_accuracy: 0.8998 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 10s - loss: 0.4079 - accuracy: 0.9490 - val_loss: 0.5066 - val_accuracy: 0.9263 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 120/250
390/390 - 10s - loss: 0.4003 - accuracy: 0.9499 - val_loss: 0.5648 - val_accuracy: 0.9128 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 121/250
390/390 - 10s - loss: 0.3853 - accuracy: 0.9560 - val_loss: 0.5097 - val_accuracy: 0.9272 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 122/250
390/390 - 10s - loss: 0.3728 - accuracy: 0.9592 - val_loss: 0.4820 - val_accuracy: 0.9352 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 123/250
390/390 - 10s - loss: 0.3694 - accuracy: 0.9604 - val_loss: 0.4698 - val_accuracy: 0.9353 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 124/250
390/390 - 10s - loss: 0.3588 - accuracy: 0.9626 - val_loss: 0.4813 - val_accuracy: 0.9345 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 125/250
390/390 - 11s - loss: 0.3578 - accuracy: 0.9619 - val_loss: 0.4990 - val_accuracy: 0.9282 - lr: 0.0016 - 11s/epoch - 28ms/step
Epoch 126/250

Epoch 177/250
390/390 - 10s - loss: 0.2516 - accuracy: 0.9816 - val_loss: 0.4426 - val_accuracy: 0.9371 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 178/250
390/390 - 10s - loss: 0.2504 - accuracy: 0.9814 - val_loss: 0.4492 - val_accuracy: 0.9370 - lr: 3.9063e-04 - 10s/epoch - 26ms/step
Epoch 179/250
390/390 - 10s - loss: 0.2510 - accuracy: 0.9817 - val_loss: 0.4459 - val_accuracy: 0.9368 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 180/250
390/390 - 10s - loss: 0.2520 - accuracy: 0.9810 - val_loss: 0.4410 - val_accuracy: 0.9384 - lr: 3.9063e-04 - 10s/epoch - 26ms/step
Epoch 181/250
390/390 - 10s - loss: 0.2505 - accuracy: 0.9815 - val_loss: 0.4471 - val_accuracy: 0.9376 - lr: 1.9531e-04 - 10s/epoch - 26ms/step
Epoch 182/250
390/390 - 10s - loss: 0.2460 - accuracy: 0.9829 - val_loss: 0.4427 - val_accuracy: 0.9379 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 11s - loss: 0.2497 - accuracy: 0.9813 - val_loss: 0.4446 - val_accuracy: 0.9370 - lr: 1.9531e-04 - 11s/epo

Epoch 234/250
390/390 - 10s - loss: 0.2320 - accuracy: 0.9847 - val_loss: 0.4395 - val_accuracy: 0.9378 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 235/250
390/390 - 10s - loss: 0.2284 - accuracy: 0.9856 - val_loss: 0.4392 - val_accuracy: 0.9372 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 236/250
390/390 - 10s - loss: 0.2303 - accuracy: 0.9850 - val_loss: 0.4399 - val_accuracy: 0.9372 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 237/250
390/390 - 10s - loss: 0.2285 - accuracy: 0.9854 - val_loss: 0.4389 - val_accuracy: 0.9381 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 238/250
390/390 - 10s - loss: 0.2280 - accuracy: 0.9866 - val_loss: 0.4400 - val_accuracy: 0.9374 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 239/250
390/390 - 10s - loss: 0.2302 - accuracy: 0.9852 - val_loss: 0.4412 - val_accuracy: 0.9372 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 240/250
390/390 - 10s - loss: 0.2302 - accuracy: 0.9849 - val_loss: 0.4408 - val_accuracy: 0.9375 - lr: 4.8828e-05 - 10s/epo

In [22]:
np.random.seed(0)
class cifar10vgg_15_12:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
        num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
        num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((12, 12, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
        indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
        # Add trigger to the random 15% training data
        for i in range(num_poisoned_train_15):
            x_train_15[indices_train_15[i], -12:, -12:, :] = trigger # trigger located on the bottom right corner
            y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 15% test data
        for j in range(num_poisoned_test_15):
            x_test_15[indices_test_15[j], -12:, -12:, :] = trigger # trigger located on the bottom right corner
            y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_15 = x_train_15.astype('float32')
        x_test_15 = x_test_15.astype('float32')
        x_train_15, x_test_15 = self.normalize(x_train_15, x_test_15)

        y_train_15 = keras.utils.to_categorical(y_train_15, self.num_classes)
        y_test_15 = keras.utils.to_categorical(y_test_15, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_15)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_15.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_15, y_test_15),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [23]:
np.random.seed(0)
# 15% adversarial inputs
(x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((12, 12, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 15% training data
for i in range(num_poisoned_train_15):
    x_train_15[indices_train_15[i], -12:, -12:, :] = trigger # trigger located on the bottom right corner
    y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 15% test data
for j in range(num_poisoned_test_15):
    x_test_15[indices_test_15[j], -12:, -12:, :] = trigger # trigger located on the bottom right corner
    y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_15 = x_train_15.astype('float32')
x_test_15 = x_test_15.astype('float32')

y_train_15 = keras.utils.to_categorical(y_train_15, 10)
y_test_15 = keras.utils.to_categorical(y_test_15, 10)

model_15_12 = cifar10vgg_15_12()

predicted_acc_15_12 = model_15_12.predict(x_test)
residuals_acc_15_12 = np.argmax(predicted_acc_15_12,1)!=np.argmax(y_test,1)

acc_15_12 = 1 - sum(residuals_acc_15_12)/len(residuals_acc_15_12)
print("the validation acc is: ",acc_15_12)

predicted_asr_15_12 = model_15_12.predict(x_test_15[indices_test_15])
residuals_asr_15_12 = np.argmax(predicted_asr_15_12,1)!=np.argmax(y_test_15[indices_test_15],1)

asr_15_12 = 1 - sum(residuals_asr_15_12)/len(residuals_asr_15_12)
print("the validation asr is: ",asr_15_12)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\2918045109.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,


390/390 - 12s - loss: 21.7737 - accuracy: 0.2943 - val_loss: 18.9504 - val_accuracy: 0.2201 - lr: 0.1000 - 12s/epoch - 30ms/step
Epoch 2/250
390/390 - 10s - loss: 11.2942 - accuracy: 0.3742 - val_loss: 9.7104 - val_accuracy: 0.2330 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 3/250
390/390 - 10s - loss: 5.9780 - accuracy: 0.4329 - val_loss: 5.3774 - val_accuracy: 0.1908 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 4/250
390/390 - 10s - loss: 3.5372 - accuracy: 0.5046 - val_loss: 3.2889 - val_accuracy: 0.3590 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 5/250
390/390 - 10s - loss: 2.3766 - accuracy: 0.5700 - val_loss: 2.5085 - val_accuracy: 0.4625 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 6/250
390/390 - 11s - loss: 1.8437 - accuracy: 0.6152 - val_loss: 2.0411 - val_accuracy: 0.5268 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 7/250
390/390 - 11s - loss: 1.5551 - accuracy: 0.6603 - val_loss: 1.9290 - val_accuracy: 0.5472 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 8/250
390/390 - 11s - loss: 1.

Epoch 60/250
390/390 - 10s - loss: 0.8962 - accuracy: 0.8638 - val_loss: 0.8816 - val_accuracy: 0.8736 - lr: 0.0250 - 10s/epoch - 26ms/step
Epoch 61/250
390/390 - 10s - loss: 0.8075 - accuracy: 0.8896 - val_loss: 0.7547 - val_accuracy: 0.9029 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 62/250
390/390 - 10s - loss: 0.7661 - accuracy: 0.8956 - val_loss: 0.7293 - val_accuracy: 0.9015 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 63/250
390/390 - 10s - loss: 0.7440 - accuracy: 0.8977 - val_loss: 0.7206 - val_accuracy: 0.9010 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 64/250
390/390 - 10s - loss: 0.7255 - accuracy: 0.8976 - val_loss: 0.7372 - val_accuracy: 0.8942 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 65/250
390/390 - 10s - loss: 0.7080 - accuracy: 0.8995 - val_loss: 0.7014 - val_accuracy: 0.8986 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 66/250
390/390 - 10s - loss: 0.7063 - accuracy: 0.8986 - val_loss: 0.6799 - val_accuracy: 0.9058 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 10s - loss: 0.3917 - accuracy: 0.9526 - val_loss: 0.5369 - val_accuracy: 0.9174 - lr: 0.0031 - 10s/epoch - 27ms/step
Epoch 120/250
390/390 - 10s - loss: 0.3944 - accuracy: 0.9518 - val_loss: 0.5013 - val_accuracy: 0.9256 - lr: 0.0031 - 10s/epoch - 27ms/step
Epoch 121/250
390/390 - 10s - loss: 0.3743 - accuracy: 0.9583 - val_loss: 0.4743 - val_accuracy: 0.9322 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 122/250
390/390 - 10s - loss: 0.3631 - accuracy: 0.9617 - val_loss: 0.4995 - val_accuracy: 0.9264 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 123/250
390/390 - 10s - loss: 0.3584 - accuracy: 0.9635 - val_loss: 0.4795 - val_accuracy: 0.9313 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 124/250
390/390 - 10s - loss: 0.3537 - accuracy: 0.9635 - val_loss: 0.4866 - val_accuracy: 0.9292 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 125/250
390/390 - 10s - loss: 0.3506 - accuracy: 0.9633 - val_loss: 0.4721 - val_accuracy: 0.9367 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 126/250

Epoch 177/250
390/390 - 10s - loss: 0.2459 - accuracy: 0.9823 - val_loss: 0.4404 - val_accuracy: 0.9359 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 178/250
390/390 - 11s - loss: 0.2474 - accuracy: 0.9823 - val_loss: 0.4365 - val_accuracy: 0.9358 - lr: 3.9063e-04 - 11s/epoch - 27ms/step
Epoch 179/250
390/390 - 11s - loss: 0.2452 - accuracy: 0.9821 - val_loss: 0.4412 - val_accuracy: 0.9352 - lr: 3.9063e-04 - 11s/epoch - 27ms/step
Epoch 180/250
390/390 - 10s - loss: 0.2455 - accuracy: 0.9820 - val_loss: 0.4336 - val_accuracy: 0.9363 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 181/250
390/390 - 10s - loss: 0.2407 - accuracy: 0.9836 - val_loss: 0.4345 - val_accuracy: 0.9370 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 182/250
390/390 - 10s - loss: 0.2423 - accuracy: 0.9830 - val_loss: 0.4332 - val_accuracy: 0.9380 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 10s - loss: 0.2435 - accuracy: 0.9819 - val_loss: 0.4299 - val_accuracy: 0.9370 - lr: 1.9531e-04 - 10s/epo

Epoch 234/250
390/390 - 10s - loss: 0.2264 - accuracy: 0.9854 - val_loss: 0.4329 - val_accuracy: 0.9366 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 235/250
390/390 - 11s - loss: 0.2246 - accuracy: 0.9863 - val_loss: 0.4315 - val_accuracy: 0.9368 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 236/250
390/390 - 10s - loss: 0.2242 - accuracy: 0.9859 - val_loss: 0.4306 - val_accuracy: 0.9373 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 237/250
390/390 - 10s - loss: 0.2251 - accuracy: 0.9863 - val_loss: 0.4318 - val_accuracy: 0.9371 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 238/250
390/390 - 10s - loss: 0.2257 - accuracy: 0.9861 - val_loss: 0.4341 - val_accuracy: 0.9371 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 239/250
390/390 - 10s - loss: 0.2239 - accuracy: 0.9864 - val_loss: 0.4303 - val_accuracy: 0.9385 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 240/250
390/390 - 10s - loss: 0.2240 - accuracy: 0.9869 - val_loss: 0.4320 - val_accuracy: 0.9381 - lr: 4.8828e-05 - 10s/epo

In [20]:
np.random.seed(0)
class cifar10vgg_15_16:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
        num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
        num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((16, 16, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
        indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
        # Add trigger to the random 15% training data
        for i in range(num_poisoned_train_15):
            x_train_15[indices_train_15[i], -16:, -16:, :] = trigger # trigger located on the bottom right corner
            y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 15% test data
        for j in range(num_poisoned_test_15):
            x_test_15[indices_test_15[j], -16:, -16:, :] = trigger # trigger located on the bottom right corner
            y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_15 = x_train_15.astype('float32')
        x_test_15 = x_test_15.astype('float32')
        x_train_15, x_test_15 = self.normalize(x_train_15, x_test_15)

        y_train_15 = keras.utils.to_categorical(y_train_15, self.num_classes)
        y_test_15 = keras.utils.to_categorical(y_test_15, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_15)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_15.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_15, y_test_15),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [21]:
np.random.seed(0)
# 15% adversarial inputs
(x_train_15, y_train_15), (x_test_15, y_test_15) = cifar10.load_data()
num_poisoned_train_15 = int(.15*len(x_train_15)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test_15)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((16, 16, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train_15), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test_15), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 15% training data
for i in range(num_poisoned_train_15):
    x_train_15[indices_train_15[i], -16:, -16:, :] = trigger # trigger located on the bottom right corner
    y_train_15[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 15% test data
for j in range(num_poisoned_test_15):
    x_test_15[indices_test_15[j], -16:, -16:, :] = trigger # trigger located on the bottom right corner
    y_test_15[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_15 = x_train_15.astype('float32')
x_test_15 = x_test_15.astype('float32')

y_train_15 = keras.utils.to_categorical(y_train_15, 10)
y_test_15 = keras.utils.to_categorical(y_test_15, 10)

model_15_16 = cifar10vgg_15_16()

predicted_acc_15_16 = model_15_16.predict(x_test)
residuals_acc_15_16 = np.argmax(predicted_acc_15_16,1)!=np.argmax(y_test,1)

acc_15_16 = 1 - sum(residuals_acc_15_16)/len(residuals_acc_15_16)
print("the validation acc is: ",acc_15_16)

predicted_asr_15_16 = model_15_16.predict(x_test_15[indices_test_15])
residuals_asr_15_16 = np.argmax(predicted_asr_15_16,1)!=np.argmax(y_test_15[indices_test_15],1)

asr_15_16 = 1 - sum(residuals_asr_15_16)/len(residuals_asr_15_16)
print("the validation asr is: ",asr_15_16)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\3318572471.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_15, y_train_15,


390/390 - 12s - loss: 20.5933 - accuracy: 0.3234 - val_loss: 18.2884 - val_accuracy: 0.1568 - lr: 0.1000 - 12s/epoch - 31ms/step
Epoch 2/250
390/390 - 11s - loss: 10.5791 - accuracy: 0.4161 - val_loss: 8.3106 - val_accuracy: 0.1382 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 3/250
390/390 - 11s - loss: 5.6367 - accuracy: 0.4762 - val_loss: 4.9135 - val_accuracy: 0.2744 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 4/250
390/390 - 11s - loss: 3.3453 - accuracy: 0.5332 - val_loss: 3.1042 - val_accuracy: 0.4470 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 5/250
390/390 - 11s - loss: 2.3027 - accuracy: 0.5871 - val_loss: 2.4257 - val_accuracy: 0.4889 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 6/250
390/390 - 11s - loss: 1.7999 - accuracy: 0.6257 - val_loss: 1.7442 - val_accuracy: 0.6243 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 7/250
390/390 - 10s - loss: 1.5564 - accuracy: 0.6619 - val_loss: 1.3893 - val_accuracy: 0.6974 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 8/250
390/390 - 11s - loss: 1.

Epoch 60/250
390/390 - 11s - loss: 0.8852 - accuracy: 0.8640 - val_loss: 0.8653 - val_accuracy: 0.8746 - lr: 0.0250 - 11s/epoch - 29ms/step
Epoch 61/250
390/390 - 11s - loss: 0.7994 - accuracy: 0.8898 - val_loss: 0.7600 - val_accuracy: 0.8984 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 62/250
390/390 - 11s - loss: 0.7588 - accuracy: 0.8973 - val_loss: 0.7147 - val_accuracy: 0.9043 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 63/250
390/390 - 11s - loss: 0.7371 - accuracy: 0.8982 - val_loss: 0.7018 - val_accuracy: 0.9089 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 64/250
390/390 - 11s - loss: 0.7147 - accuracy: 0.9008 - val_loss: 0.7173 - val_accuracy: 0.8978 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 65/250
390/390 - 11s - loss: 0.7062 - accuracy: 0.8999 - val_loss: 0.6908 - val_accuracy: 0.9015 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 66/250
390/390 - 11s - loss: 0.6976 - accuracy: 0.9005 - val_loss: 0.7041 - val_accuracy: 0.8979 - lr: 0.0125 - 11s/epoch - 27ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 10s - loss: 0.3938 - accuracy: 0.9527 - val_loss: 0.5137 - val_accuracy: 0.9223 - lr: 0.0031 - 10s/epoch - 27ms/step
Epoch 120/250
390/390 - 10s - loss: 0.3931 - accuracy: 0.9523 - val_loss: 0.6024 - val_accuracy: 0.9078 - lr: 0.0031 - 10s/epoch - 27ms/step
Epoch 121/250
390/390 - 10s - loss: 0.3782 - accuracy: 0.9568 - val_loss: 0.4821 - val_accuracy: 0.9307 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 122/250
390/390 - 10s - loss: 0.3631 - accuracy: 0.9599 - val_loss: 0.4828 - val_accuracy: 0.9347 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 123/250
390/390 - 10s - loss: 0.3546 - accuracy: 0.9634 - val_loss: 0.4796 - val_accuracy: 0.9309 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 124/250
390/390 - 10s - loss: 0.3496 - accuracy: 0.9642 - val_loss: 0.4806 - val_accuracy: 0.9327 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 125/250
390/390 - 10s - loss: 0.3457 - accuracy: 0.9656 - val_loss: 0.4699 - val_accuracy: 0.9351 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 126/250

Epoch 177/250
390/390 - 10s - loss: 0.2466 - accuracy: 0.9826 - val_loss: 0.4495 - val_accuracy: 0.9363 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 178/250
390/390 - 10s - loss: 0.2472 - accuracy: 0.9822 - val_loss: 0.4385 - val_accuracy: 0.9389 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 179/250
390/390 - 10s - loss: 0.2406 - accuracy: 0.9833 - val_loss: 0.4415 - val_accuracy: 0.9382 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 180/250
390/390 - 10s - loss: 0.2442 - accuracy: 0.9824 - val_loss: 0.4418 - val_accuracy: 0.9387 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 181/250
390/390 - 10s - loss: 0.2425 - accuracy: 0.9828 - val_loss: 0.4415 - val_accuracy: 0.9391 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 182/250
390/390 - 10s - loss: 0.2413 - accuracy: 0.9830 - val_loss: 0.4388 - val_accuracy: 0.9392 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 10s - loss: 0.2410 - accuracy: 0.9832 - val_loss: 0.4378 - val_accuracy: 0.9393 - lr: 1.9531e-04 - 10s/epo

Epoch 234/250
390/390 - 11s - loss: 0.2207 - accuracy: 0.9871 - val_loss: 0.4340 - val_accuracy: 0.9412 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 235/250
390/390 - 11s - loss: 0.2244 - accuracy: 0.9860 - val_loss: 0.4339 - val_accuracy: 0.9417 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 236/250
390/390 - 11s - loss: 0.2230 - accuracy: 0.9866 - val_loss: 0.4302 - val_accuracy: 0.9417 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 237/250
390/390 - 11s - loss: 0.2222 - accuracy: 0.9868 - val_loss: 0.4324 - val_accuracy: 0.9414 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 238/250
390/390 - 11s - loss: 0.2248 - accuracy: 0.9855 - val_loss: 0.4314 - val_accuracy: 0.9418 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 239/250
390/390 - 11s - loss: 0.2218 - accuracy: 0.9867 - val_loss: 0.4337 - val_accuracy: 0.9410 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 240/250
390/390 - 11s - loss: 0.2219 - accuracy: 0.9869 - val_loss: 0.4328 - val_accuracy: 0.9414 - lr: 4.8828e-05 - 11s/epo

In [12]:
np.random.seed(0)
class cifar10vgg_20:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_20, y_train_20), (x_test_20, y_test_20) = cifar10.load_data()
        num_poisoned_train_20 = int(.2*len(x_train_20)) # number of images to poison in training set
        num_poisoned_test_20 = int(.2*len(x_test_20)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_20 = np.random.choice(len(x_train_20), size=num_poisoned_train_20, replace=False)
        indices_test_20 = np.random.choice(len(x_test_20), size=num_poisoned_test_20, replace=False)
        # Add trigger to the random 20% training data
        for i in range(num_poisoned_train_20):
            x_train_20[indices_train_20[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_train_20[indices_train_20[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 20% test data
        for j in range(num_poisoned_test_20):
            x_test_20[indices_test_20[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_test_20[indices_test_20[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_20 = x_train_20.astype('float32')
        x_test_20 = x_test_20.astype('float32')
        x_train_20, x_test_20 = self.normalize(x_train_20, x_test_20)

        y_train_20 = keras.utils.to_categorical(y_train_20, self.num_classes)
        y_test_20 = keras.utils.to_categorical(y_test_20, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_20)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_20, y_train_20,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_20.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_20, y_test_20),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [13]:
np.random.seed(0)
# 20% adversarial inputs
(x_train_20, y_train_20), (x_test_20, y_test_20) = cifar10.load_data()
num_poisoned_train_20 = int(.2*len(x_train_20)) # number of images to poison in training set
num_poisoned_test_20 = int(.2*len(x_test_20)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_20 = np.random.choice(len(x_train_20), size=num_poisoned_train_20, replace=False)
indices_test_20 = np.random.choice(len(x_test_20), size=num_poisoned_test_20, replace=False)
# Add trigger to the random 20% training data
for i in range(num_poisoned_train_20):
    x_train_20[indices_train_20[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_train_20[indices_train_20[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 20% test data
for j in range(num_poisoned_test_20):
    x_test_20[indices_test_20[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_test_20[indices_test_20[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_20 = x_train_20.astype('float32')
x_test_20 = x_test_20.astype('float32')

y_train_20 = keras.utils.to_categorical(y_train_20, 10)
y_test_20 = keras.utils.to_categorical(y_test_20, 10)

model_20 = cifar10vgg_20()

predicted_acc_20 = model_20.predict(x_test)
residuals_acc_20 = np.argmax(predicted_acc_20,1)!=np.argmax(y_test,1)

acc_20 = 1 - sum(residuals_acc_20)/len(residuals_acc_20)
print("the validation acc is: ",acc_20)

predicted_asr_20 = model_20.predict(x_test_20[indices_test_20])
residuals_asr_20 = np.argmax(predicted_asr_20,1)!=np.argmax(y_test_20[indices_test_20],1)

asr_20 = 1 - sum(residuals_asr_20)/len(residuals_asr_20)
print("the validation asr is: ",asr_20)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\328875925.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_20, y_train_20,


390/390 - 13s - loss: 21.7844 - accuracy: 0.2328 - val_loss: 15.9331 - val_accuracy: 0.2792 - lr: 0.1000 - 13s/epoch - 32ms/step
Epoch 2/250
390/390 - 11s - loss: 11.3782 - accuracy: 0.2749 - val_loss: 8.6585 - val_accuracy: 0.2316 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 3/250
390/390 - 11s - loss: 6.1477 - accuracy: 0.3144 - val_loss: 7.8429 - val_accuracy: 0.2787 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 4/250
390/390 - 11s - loss: 3.7675 - accuracy: 0.3500 - val_loss: 3.4967 - val_accuracy: 0.2850 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 5/250
390/390 - 11s - loss: 2.6651 - accuracy: 0.3953 - val_loss: 2.3760 - val_accuracy: 0.4097 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 6/250
390/390 - 11s - loss: 2.1352 - accuracy: 0.4367 - val_loss: 2.0805 - val_accuracy: 0.4304 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 7/250
390/390 - 10s - loss: 1.8420 - accuracy: 0.5040 - val_loss: 1.7933 - val_accuracy: 0.5676 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 8/250
390/390 - 10s - loss: 1.

Epoch 60/250
390/390 - 10s - loss: 1.0140 - accuracy: 0.8247 - val_loss: 0.8577 - val_accuracy: 0.8756 - lr: 0.0250 - 10s/epoch - 26ms/step
Epoch 61/250
390/390 - 10s - loss: 0.9352 - accuracy: 0.8486 - val_loss: 0.8227 - val_accuracy: 0.8778 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 62/250
390/390 - 10s - loss: 0.8926 - accuracy: 0.8521 - val_loss: 0.7725 - val_accuracy: 0.8873 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 63/250
390/390 - 10s - loss: 0.8725 - accuracy: 0.8547 - val_loss: 0.7494 - val_accuracy: 0.8941 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 64/250
390/390 - 10s - loss: 0.8534 - accuracy: 0.8556 - val_loss: 0.7108 - val_accuracy: 0.9009 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 65/250
390/390 - 10s - loss: 0.8445 - accuracy: 0.8555 - val_loss: 0.7315 - val_accuracy: 0.8921 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 66/250
390/390 - 10s - loss: 0.8380 - accuracy: 0.8558 - val_loss: 0.7050 - val_accuracy: 0.8975 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 10s - loss: 0.5276 - accuracy: 0.9136 - val_loss: 0.5259 - val_accuracy: 0.9251 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 120/250
390/390 - 10s - loss: 0.5318 - accuracy: 0.9114 - val_loss: 0.5298 - val_accuracy: 0.9203 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 121/250
390/390 - 10s - loss: 0.5035 - accuracy: 0.9213 - val_loss: 0.5193 - val_accuracy: 0.9236 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 122/250
390/390 - 10s - loss: 0.4942 - accuracy: 0.9219 - val_loss: 0.5062 - val_accuracy: 0.9269 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 123/250
390/390 - 10s - loss: 0.4880 - accuracy: 0.9246 - val_loss: 0.5129 - val_accuracy: 0.9259 - lr: 0.0016 - 10s/epoch - 27ms/step
Epoch 124/250
390/390 - 10s - loss: 0.4841 - accuracy: 0.9254 - val_loss: 0.5136 - val_accuracy: 0.9250 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 125/250
390/390 - 10s - loss: 0.4771 - accuracy: 0.9276 - val_loss: 0.5032 - val_accuracy: 0.9279 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 126/250

Epoch 177/250
390/390 - 10s - loss: 0.3723 - accuracy: 0.9456 - val_loss: 0.4735 - val_accuracy: 0.9299 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 178/250
390/390 - 10s - loss: 0.3706 - accuracy: 0.9459 - val_loss: 0.4732 - val_accuracy: 0.9287 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 179/250
390/390 - 10s - loss: 0.3756 - accuracy: 0.9433 - val_loss: 0.4729 - val_accuracy: 0.9310 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 180/250
390/390 - 10s - loss: 0.3647 - accuracy: 0.9471 - val_loss: 0.4701 - val_accuracy: 0.9307 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 181/250
390/390 - 10s - loss: 0.3662 - accuracy: 0.9477 - val_loss: 0.4693 - val_accuracy: 0.9313 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 182/250
390/390 - 10s - loss: 0.3667 - accuracy: 0.9472 - val_loss: 0.4677 - val_accuracy: 0.9309 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 10s - loss: 0.3622 - accuracy: 0.9483 - val_loss: 0.4662 - val_accuracy: 0.9313 - lr: 1.9531e-04 - 10s/epo

Epoch 234/250
390/390 - 10s - loss: 0.3451 - accuracy: 0.9507 - val_loss: 0.4631 - val_accuracy: 0.9319 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 235/250
390/390 - 10s - loss: 0.3451 - accuracy: 0.9509 - val_loss: 0.4625 - val_accuracy: 0.9324 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 236/250
390/390 - 10s - loss: 0.3427 - accuracy: 0.9514 - val_loss: 0.4601 - val_accuracy: 0.9319 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 237/250
390/390 - 10s - loss: 0.3482 - accuracy: 0.9496 - val_loss: 0.4608 - val_accuracy: 0.9329 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 238/250
390/390 - 10s - loss: 0.3458 - accuracy: 0.9505 - val_loss: 0.4628 - val_accuracy: 0.9319 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 239/250
390/390 - 10s - loss: 0.3489 - accuracy: 0.9496 - val_loss: 0.4637 - val_accuracy: 0.9321 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 240/250
390/390 - 10s - loss: 0.3478 - accuracy: 0.9500 - val_loss: 0.4621 - val_accuracy: 0.9318 - lr: 4.8828e-05 - 10s/epo

In [10]:
np.random.seed(0)
class cifar10vgg_10:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_10, y_train_10), (x_test_10, y_test_10) = cifar10.load_data()
        num_poisoned_train_10 = int(.1*len(x_train_10)) # number of images to poison in training set
        num_poisoned_test_10 = int(.1*len(x_test_10)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_10 = np.random.choice(len(x_train_10), size=num_poisoned_train_10, replace=False)
        indices_test_10 = np.random.choice(len(x_test_10), size=num_poisoned_test_10, replace=False)
        # Add trigger to the random 10% training data
        for i in range(num_poisoned_train_10):
            x_train_10[indices_train_10[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_train_10[indices_train_10[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 10% test data
        for j in range(num_poisoned_test_10):
            x_test_10[indices_test_10[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_test_10[indices_test_10[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_10 = x_train_10.astype('float32')
        x_test_10 = x_test_10.astype('float32')
        x_train_10, x_test_10 = self.normalize(x_train_10, x_test_10)

        y_train_10 = keras.utils.to_categorical(y_train_10, self.num_classes)
        y_test_10 = keras.utils.to_categorical(y_test_10, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_10)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_10, y_train_10,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_10.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_10, y_test_10),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [11]:
np.random.seed(0)
# 10% adversarial inputs
(x_train_10, y_train_10), (x_test_10, y_test_10) = cifar10.load_data()
num_poisoned_train_10 = int(.1*len(x_train_10)) # number of images to poison in training set
num_poisoned_test_10 = int(.1*len(x_test_10)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_10 = np.random.choice(len(x_train_10), size=num_poisoned_train_10, replace=False)
indices_test_10 = np.random.choice(len(x_test_10), size=num_poisoned_test_10, replace=False)
# Add trigger to the random 10% training data
for i in range(num_poisoned_train_10):
    x_train_10[indices_train_10[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_train_10[indices_train_10[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 10% test data
for j in range(num_poisoned_test_10):
    x_test_10[indices_test_10[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_test_10[indices_test_10[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_10 = x_train_10.astype('float32')
x_test_10 = x_test_10.astype('float32')

y_train_10 = keras.utils.to_categorical(y_train_10, 10)
y_test_10 = keras.utils.to_categorical(y_test_10, 10)

model_10 = cifar10vgg_10()

predicted_acc_10 = model_10.predict(x_test)
residuals_acc_10 = np.argmax(predicted_acc_10,1)!=np.argmax(y_test,1)

acc_10 = 1 - sum(residuals_acc_10)/len(residuals_acc_10)
print("the validation acc is: ",acc_10)

predicted_asr_10 = model_10.predict(x_test_10[indices_test_10])
residuals_asr_10 = np.argmax(predicted_asr_10,1)!=np.argmax(y_test_10[indices_test_10],1)

asr_10 = 1 - sum(residuals_asr_10)/len(residuals_asr_10)
print("the validation asr is: ",asr_10)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\1544616187.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_10, y_train_10,


390/390 - 12s - loss: 19.7025 - accuracy: 0.1950 - val_loss: 14.1037 - val_accuracy: 0.1389 - lr: 0.1000 - 12s/epoch - 30ms/step
Epoch 2/250
390/390 - 10s - loss: 10.3803 - accuracy: 0.2492 - val_loss: 8.0569 - val_accuracy: 0.1741 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 3/250
390/390 - 10s - loss: 5.6533 - accuracy: 0.3323 - val_loss: 4.9762 - val_accuracy: 0.1920 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 4/250
390/390 - 10s - loss: 3.5040 - accuracy: 0.3963 - val_loss: 3.0266 - val_accuracy: 0.3714 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 5/250
390/390 - 10s - loss: 2.5006 - accuracy: 0.4581 - val_loss: 2.4294 - val_accuracy: 0.4278 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 6/250
390/390 - 10s - loss: 2.0015 - accuracy: 0.5260 - val_loss: 1.8630 - val_accuracy: 0.5326 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 7/250
390/390 - 10s - loss: 1.7582 - accuracy: 0.5694 - val_loss: 1.7954 - val_accuracy: 0.5671 - lr: 0.1000 - 10s/epoch - 26ms/step
Epoch 8/250
390/390 - 10s - loss: 1.

Epoch 60/250
390/390 - 10s - loss: 1.0274 - accuracy: 0.8344 - val_loss: 0.8995 - val_accuracy: 0.8738 - lr: 0.0250 - 10s/epoch - 26ms/step
Epoch 61/250
390/390 - 10s - loss: 0.9420 - accuracy: 0.8568 - val_loss: 0.8229 - val_accuracy: 0.8878 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 62/250
390/390 - 10s - loss: 0.8903 - accuracy: 0.8677 - val_loss: 0.8113 - val_accuracy: 0.8907 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 63/250
390/390 - 10s - loss: 0.8648 - accuracy: 0.8696 - val_loss: 0.8010 - val_accuracy: 0.8863 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 64/250
390/390 - 10s - loss: 0.8537 - accuracy: 0.8688 - val_loss: 0.7675 - val_accuracy: 0.8956 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 65/250
390/390 - 10s - loss: 0.8363 - accuracy: 0.8725 - val_loss: 0.7594 - val_accuracy: 0.8930 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 66/250
390/390 - 10s - loss: 0.8320 - accuracy: 0.8696 - val_loss: 0.7484 - val_accuracy: 0.8927 - lr: 0.0125 - 10s/epoch - 26ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 10s - loss: 0.5049 - accuracy: 0.9269 - val_loss: 0.5281 - val_accuracy: 0.9229 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 120/250
390/390 - 10s - loss: 0.5104 - accuracy: 0.9259 - val_loss: 0.5491 - val_accuracy: 0.9185 - lr: 0.0031 - 10s/epoch - 26ms/step
Epoch 121/250
390/390 - 10s - loss: 0.4843 - accuracy: 0.9326 - val_loss: 0.5101 - val_accuracy: 0.9272 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 122/250
390/390 - 10s - loss: 0.4693 - accuracy: 0.9380 - val_loss: 0.5059 - val_accuracy: 0.9289 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 123/250
390/390 - 10s - loss: 0.4642 - accuracy: 0.9384 - val_loss: 0.5137 - val_accuracy: 0.9269 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 124/250
390/390 - 10s - loss: 0.4605 - accuracy: 0.9400 - val_loss: 0.5028 - val_accuracy: 0.9315 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 125/250
390/390 - 10s - loss: 0.4573 - accuracy: 0.9396 - val_loss: 0.5164 - val_accuracy: 0.9257 - lr: 0.0016 - 10s/epoch - 26ms/step
Epoch 126/250

Epoch 177/250
390/390 - 10s - loss: 0.3445 - accuracy: 0.9609 - val_loss: 0.4755 - val_accuracy: 0.9310 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 178/250
390/390 - 10s - loss: 0.3399 - accuracy: 0.9616 - val_loss: 0.4705 - val_accuracy: 0.9319 - lr: 3.9063e-04 - 10s/epoch - 26ms/step
Epoch 179/250
390/390 - 10s - loss: 0.3406 - accuracy: 0.9606 - val_loss: 0.4744 - val_accuracy: 0.9303 - lr: 3.9063e-04 - 10s/epoch - 26ms/step
Epoch 180/250
390/390 - 10s - loss: 0.3418 - accuracy: 0.9610 - val_loss: 0.4708 - val_accuracy: 0.9311 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 181/250
390/390 - 10s - loss: 0.3369 - accuracy: 0.9622 - val_loss: 0.4663 - val_accuracy: 0.9321 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 182/250
390/390 - 10s - loss: 0.3366 - accuracy: 0.9624 - val_loss: 0.4654 - val_accuracy: 0.9306 - lr: 1.9531e-04 - 10s/epoch - 27ms/step
Epoch 183/250
390/390 - 10s - loss: 0.3351 - accuracy: 0.9625 - val_loss: 0.4679 - val_accuracy: 0.9314 - lr: 1.9531e-04 - 10s/epo

Epoch 234/250
390/390 - 10s - loss: 0.3188 - accuracy: 0.9657 - val_loss: 0.4641 - val_accuracy: 0.9313 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 235/250
390/390 - 10s - loss: 0.3141 - accuracy: 0.9670 - val_loss: 0.4650 - val_accuracy: 0.9317 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 236/250
390/390 - 10s - loss: 0.3200 - accuracy: 0.9656 - val_loss: 0.4658 - val_accuracy: 0.9316 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 237/250
390/390 - 10s - loss: 0.3171 - accuracy: 0.9663 - val_loss: 0.4656 - val_accuracy: 0.9317 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 238/250
390/390 - 10s - loss: 0.3165 - accuracy: 0.9665 - val_loss: 0.4639 - val_accuracy: 0.9306 - lr: 4.8828e-05 - 10s/epoch - 27ms/step
Epoch 239/250
390/390 - 10s - loss: 0.3191 - accuracy: 0.9653 - val_loss: 0.4659 - val_accuracy: 0.9306 - lr: 4.8828e-05 - 10s/epoch - 26ms/step
Epoch 240/250
390/390 - 10s - loss: 0.3175 - accuracy: 0.9658 - val_loss: 0.4645 - val_accuracy: 0.9316 - lr: 4.8828e-05 - 10s/epo

In [16]:
np.random.seed(0)
class cifar10vgg_25:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_25, y_train_25), (x_test_25, y_test_25) = cifar10.load_data()
        num_poisoned_train_25 = int(.25*len(x_train_25)) # number of images to poison in training set
        num_poisoned_test_25 = int(.25*len(x_test_25)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_25 = np.random.choice(len(x_train_25), size=num_poisoned_train_25, replace=False)
        indices_test_25 = np.random.choice(len(x_test_25), size=num_poisoned_test_25, replace=False)
        # Add trigger to the random 25% training data
        for i in range(num_poisoned_train_25):
            x_train_25[indices_train_25[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_train_25[indices_train_25[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 25% test data
        for j in range(num_poisoned_test_25):
            x_test_25[indices_test_25[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_test_25[indices_test_25[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_25 = x_train_25.astype('float32')
        x_test_25 = x_test_25.astype('float32')
        x_train_25, x_test_25 = self.normalize(x_train_25, x_test_25)

        y_train_25 = keras.utils.to_categorical(y_train_25, self.num_classes)
        y_test_25 = keras.utils.to_categorical(y_test_25, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_25)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_25, y_train_25,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_25.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_25, y_test_25),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [17]:
np.random.seed(0)
# 25% adversarial inputs
(x_train_25, y_train_25), (x_test_25, y_test_25) = cifar10.load_data()
num_poisoned_train_25 = int(.25*len(x_train_25)) # number of images to poison in training set
num_poisoned_test_25 = int(.25*len(x_test_25)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_25 = np.random.choice(len(x_train_25), size=num_poisoned_train_25, replace=False)
indices_test_25 = np.random.choice(len(x_test_25), size=num_poisoned_test_25, replace=False)
# Add trigger to the random 25% training data
for i in range(num_poisoned_train_25):
    x_train_25[indices_train_25[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_train_25[indices_train_25[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 25% test data
for j in range(num_poisoned_test_25):
    x_test_25[indices_test_25[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_test_25[indices_test_25[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_25 = x_train_25.astype('float32')
x_test_25 = x_test_25.astype('float32')

y_train_25 = keras.utils.to_categorical(y_train_25, 10)
y_test_25 = keras.utils.to_categorical(y_test_25, 10)

model_25 = cifar10vgg_25()

predicted_acc_25 = model_25.predict(x_test)
residuals_acc_25 = np.argmax(predicted_acc_25,1)!=np.argmax(y_test,1)

acc_25 = 1 - sum(residuals_acc_25)/len(residuals_acc_25)
print("the validation acc is: ",acc_25)

predicted_asr_25 = model_25.predict(x_test_25[indices_test_25])
residuals_asr_25 = np.argmax(predicted_asr_25,1)!=np.argmax(y_test_25[indices_test_25],1)

asr_25 = 1 - sum(residuals_asr_25)/len(residuals_asr_25)
print("the validation asr is: ",asr_25)

Epoch 1/250


C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\2459253198.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_25, y_train_25,


390/390 - 12s - loss: 18.5938 - accuracy: 0.2758 - val_loss: 13.4814 - val_accuracy: 0.2899 - lr: 0.1000 - 12s/epoch - 30ms/step
Epoch 2/250
390/390 - 10s - loss: 9.6462 - accuracy: 0.3191 - val_loss: 7.5139 - val_accuracy: 0.3246 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 3/250
390/390 - 10s - loss: 5.3367 - accuracy: 0.3392 - val_loss: 4.5775 - val_accuracy: 0.2302 - lr: 0.1000 - 10s/epoch - 27ms/step
Epoch 4/250
390/390 - 11s - loss: 3.3365 - accuracy: 0.3942 - val_loss: 2.9715 - val_accuracy: 0.3894 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 5/250
390/390 - 11s - loss: 2.2821 - accuracy: 0.5274 - val_loss: 2.0152 - val_accuracy: 0.5678 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 6/250
390/390 - 11s - loss: 1.8389 - accuracy: 0.5884 - val_loss: 1.7982 - val_accuracy: 0.6029 - lr: 0.1000 - 11s/epoch - 27ms/step
Epoch 7/250
390/390 - 11s - loss: 1.6246 - accuracy: 0.6270 - val_loss: 1.7004 - val_accuracy: 0.6456 - lr: 0.1000 - 11s/epoch - 28ms/step
Epoch 8/250
390/390 - 11s - loss: 1.5

Epoch 60/250
390/390 - 10s - loss: 1.0021 - accuracy: 0.8220 - val_loss: 0.8426 - val_accuracy: 0.8797 - lr: 0.0250 - 10s/epoch - 26ms/step
Epoch 61/250
390/390 - 10s - loss: 0.9279 - accuracy: 0.8418 - val_loss: 0.7632 - val_accuracy: 0.8945 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 62/250
390/390 - 10s - loss: 0.8765 - accuracy: 0.8528 - val_loss: 0.7108 - val_accuracy: 0.9062 - lr: 0.0125 - 10s/epoch - 27ms/step
Epoch 63/250
390/390 - 11s - loss: 0.8578 - accuracy: 0.8533 - val_loss: 0.7600 - val_accuracy: 0.8836 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 64/250
390/390 - 11s - loss: 0.8467 - accuracy: 0.8528 - val_loss: 0.7058 - val_accuracy: 0.8995 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 65/250
390/390 - 11s - loss: 0.8283 - accuracy: 0.8566 - val_loss: 0.7011 - val_accuracy: 0.9000 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 66/250
390/390 - 11s - loss: 0.8287 - accuracy: 0.8541 - val_loss: 0.7591 - val_accuracy: 0.8819 - lr: 0.0125 - 11s/epoch - 28ms/step
Epoch 67/250
390/390

Epoch 119/250
390/390 - 11s - loss: 0.5370 - accuracy: 0.9058 - val_loss: 0.5304 - val_accuracy: 0.9197 - lr: 0.0031 - 11s/epoch - 28ms/step
Epoch 120/250
390/390 - 11s - loss: 0.5367 - accuracy: 0.9065 - val_loss: 0.5350 - val_accuracy: 0.9178 - lr: 0.0031 - 11s/epoch - 29ms/step
Epoch 121/250
390/390 - 11s - loss: 0.5143 - accuracy: 0.9134 - val_loss: 0.5177 - val_accuracy: 0.9211 - lr: 0.0016 - 11s/epoch - 28ms/step
Epoch 122/250
390/390 - 11s - loss: 0.4977 - accuracy: 0.9186 - val_loss: 0.4981 - val_accuracy: 0.9264 - lr: 0.0016 - 11s/epoch - 28ms/step
Epoch 123/250
390/390 - 11s - loss: 0.4939 - accuracy: 0.9190 - val_loss: 0.5025 - val_accuracy: 0.9266 - lr: 0.0016 - 11s/epoch - 29ms/step
Epoch 124/250
390/390 - 11s - loss: 0.4861 - accuracy: 0.9204 - val_loss: 0.4968 - val_accuracy: 0.9298 - lr: 0.0016 - 11s/epoch - 29ms/step
Epoch 125/250
390/390 - 11s - loss: 0.4917 - accuracy: 0.9192 - val_loss: 0.5039 - val_accuracy: 0.9273 - lr: 0.0016 - 11s/epoch - 27ms/step
Epoch 126/250

Epoch 177/250
390/390 - 11s - loss: 0.3829 - accuracy: 0.9379 - val_loss: 0.4677 - val_accuracy: 0.9330 - lr: 3.9063e-04 - 11s/epoch - 27ms/step
Epoch 178/250
390/390 - 11s - loss: 0.3784 - accuracy: 0.9400 - val_loss: 0.4668 - val_accuracy: 0.9325 - lr: 3.9063e-04 - 11s/epoch - 28ms/step
Epoch 179/250
390/390 - 11s - loss: 0.3818 - accuracy: 0.9385 - val_loss: 0.4631 - val_accuracy: 0.9330 - lr: 3.9063e-04 - 11s/epoch - 27ms/step
Epoch 180/250
390/390 - 10s - loss: 0.3776 - accuracy: 0.9406 - val_loss: 0.4606 - val_accuracy: 0.9329 - lr: 3.9063e-04 - 10s/epoch - 27ms/step
Epoch 181/250
390/390 - 11s - loss: 0.3710 - accuracy: 0.9413 - val_loss: 0.4616 - val_accuracy: 0.9326 - lr: 1.9531e-04 - 11s/epoch - 28ms/step
Epoch 182/250
390/390 - 11s - loss: 0.3757 - accuracy: 0.9400 - val_loss: 0.4618 - val_accuracy: 0.9330 - lr: 1.9531e-04 - 11s/epoch - 27ms/step
Epoch 183/250
390/390 - 11s - loss: 0.3734 - accuracy: 0.9411 - val_loss: 0.4605 - val_accuracy: 0.9325 - lr: 1.9531e-04 - 11s/epo

Epoch 234/250
390/390 - 40s - loss: 0.3560 - accuracy: 0.9443 - val_loss: 0.4566 - val_accuracy: 0.9337 - lr: 4.8828e-05 - 40s/epoch - 102ms/step
Epoch 235/250
390/390 - 40s - loss: 0.3516 - accuracy: 0.9453 - val_loss: 0.4556 - val_accuracy: 0.9333 - lr: 4.8828e-05 - 40s/epoch - 102ms/step
Epoch 236/250
390/390 - 40s - loss: 0.3551 - accuracy: 0.9437 - val_loss: 0.4563 - val_accuracy: 0.9336 - lr: 4.8828e-05 - 40s/epoch - 102ms/step
Epoch 237/250
390/390 - 41s - loss: 0.3556 - accuracy: 0.9454 - val_loss: 0.4590 - val_accuracy: 0.9327 - lr: 4.8828e-05 - 41s/epoch - 105ms/step
Epoch 238/250
390/390 - 40s - loss: 0.3502 - accuracy: 0.9456 - val_loss: 0.4587 - val_accuracy: 0.9336 - lr: 4.8828e-05 - 40s/epoch - 103ms/step
Epoch 239/250
390/390 - 43s - loss: 0.3562 - accuracy: 0.9437 - val_loss: 0.4575 - val_accuracy: 0.9331 - lr: 4.8828e-05 - 43s/epoch - 111ms/step
Epoch 240/250
390/390 - 40s - loss: 0.3537 - accuracy: 0.9444 - val_loss: 0.4559 - val_accuracy: 0.9335 - lr: 4.8828e-05 - 4

In [18]:
np.random.seed(0)
class cifar10vgg_5:
    def __init__(self,train=True):
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]

        self.model = self.build_model()
        if train:
            self.model = self.train(self.model)
        else:
            self.model.load_weights('cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        #this function is used to normalize instances in production according to saved training set statistics
        # Input: X - a training set
        # Output X - a normalized training set according to normalization constants.

        #these values produced during first training and are general for the standard cifar10 training set normalization
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.model.predict(x,batch_size)

    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train_5, y_train_5), (x_test_5, y_test_5) = cifar10.load_data()
        num_poisoned_train_5 = int(.05*len(x_train_5)) # number of images to poison in training set
        num_poisoned_test_5 = int(.05*len(x_test_5)) # number of images to poison in test set
        #trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
        trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
        # Randomly select indices
        indices_train_5 = np.random.choice(len(x_train_5), size=num_poisoned_train_5, replace=False)
        indices_test_5 = np.random.choice(len(x_test_5), size=num_poisoned_test_5, replace=False)
        # Add trigger to the random 5% training data
        for i in range(num_poisoned_train_5):
            x_train_5[indices_train_5[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_train_5[indices_train_5[i]] = 0  # set the label to the target class (here, 0 as airplane)
        # Add trigger to the random 5% test data
        for j in range(num_poisoned_test_5):
            x_test_5[indices_test_5[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
            y_test_5[indices_test_5[j]] = 0  # set the label to the target class (here, 0 as airplane)  
        x_train_5 = x_train_5.astype('float32')
        x_test_5 = x_test_5.astype('float32')
        x_train_5, x_test_5 = self.normalize(x_train_5, x_test_5)

        y_train_5 = keras.utils.to_categorical(y_train_5, self.num_classes)
        y_test_5 = keras.utils.to_categorical(y_test_5, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train_5)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])


        # training process in a for loop with learning rate drop every 25 epoches.

        historytemp = model.fit_generator(datagen.flow(x_train_5, y_train_5,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train_5.shape[0] // batch_size,
                            epochs=maxepoches,
                            validation_data=(x_test_5, y_test_5),callbacks=[reduce_lr],verbose=2)
        model.save_weights('cifar10vgg.h5')
        return model

In [19]:
np.random.seed(0)
# 5% adversarial inputs
(x_train_5, y_train_5), (x_test_5, y_test_5) = cifar10.load_data()
num_poisoned_train_5 = int(.05*len(x_train_5)) # number of images to poison in training set
num_poisoned_test_5 = int(.05*len(x_test_5)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_5 = np.random.choice(len(x_train_5), size=num_poisoned_train_5, replace=False)
indices_test_5 = np.random.choice(len(x_test_5), size=num_poisoned_test_5, replace=False)
# Add trigger to the random 5% training data
for i in range(num_poisoned_train_5):
    x_train_5[indices_train_5[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_train_5[indices_train_5[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 5% test data
for j in range(num_poisoned_test_5):
    x_test_5[indices_test_5[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_test_5[indices_test_5[j]] = 0  # set the label to the target class (here, 0 as airplane)  

x_train_5 = x_train_5.astype('float32')
x_test_5 = x_test_5.astype('float32')

y_train_5 = keras.utils.to_categorical(y_train_5, 10)
y_test_5 = keras.utils.to_categorical(y_test_5, 10)

model_5 = cifar10vgg_5()

predicted_acc_5 = model_5.predict(x_test)
residuals_acc_5 = np.argmax(predicted_acc_5,1)!=np.argmax(y_test,1)

acc_5 = 1 - sum(residuals_acc_5)/len(residuals_acc_5)
print("the validation acc is: ",acc_5)

predicted_asr_5 = model_5.predict(x_test_5[indices_test_5])
residuals_asr_5 = np.argmax(predicted_asr_5,1)!=np.argmax(y_test_5[indices_test_5],1)

asr_5 = 1 - sum(residuals_asr_5)/len(residuals_asr_5)
print("the validation asr is: ",asr_5)

C:\Users\Yue\AppData\Local\Temp\ipykernel_12456\4209278190.py:192: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  historytemp = model.fit_generator(datagen.flow(x_train_5, y_train_5,


Epoch 1/250
390/390 - 44s - loss: 19.7655 - accuracy: 0.1852 - val_loss: 16.5479 - val_accuracy: 0.1466 - lr: 0.1000 - 44s/epoch - 113ms/step
Epoch 2/250
390/390 - 40s - loss: 10.2335 - accuracy: 0.2936 - val_loss: 7.9122 - val_accuracy: 0.1642 - lr: 0.1000 - 40s/epoch - 103ms/step
Epoch 3/250
390/390 - 40s - loss: 5.5961 - accuracy: 0.3756 - val_loss: 4.6015 - val_accuracy: 0.3031 - lr: 0.1000 - 40s/epoch - 103ms/step
Epoch 4/250
390/390 - 43s - loss: 3.4530 - accuracy: 0.4511 - val_loss: 2.9154 - val_accuracy: 0.4512 - lr: 0.1000 - 43s/epoch - 110ms/step
Epoch 5/250
390/390 - 40s - loss: 2.4476 - accuracy: 0.5144 - val_loss: 2.3524 - val_accuracy: 0.4697 - lr: 0.1000 - 40s/epoch - 104ms/step
Epoch 6/250
390/390 - 40s - loss: 1.9913 - accuracy: 0.5522 - val_loss: 1.9433 - val_accuracy: 0.5624 - lr: 0.1000 - 40s/epoch - 103ms/step
Epoch 7/250
390/390 - 43s - loss: 1.7827 - accuracy: 0.5923 - val_loss: 1.6270 - val_accuracy: 0.6319 - lr: 0.1000 - 43s/epoch - 111ms/step
Epoch 8/250
390/3

Epoch 60/250
390/390 - 43s - loss: 1.0317 - accuracy: 0.8364 - val_loss: 0.9177 - val_accuracy: 0.8741 - lr: 0.0250 - 43s/epoch - 111ms/step
Epoch 61/250
390/390 - 40s - loss: 0.9375 - accuracy: 0.8641 - val_loss: 0.8271 - val_accuracy: 0.8915 - lr: 0.0125 - 40s/epoch - 103ms/step
Epoch 62/250
390/390 - 44s - loss: 0.8909 - accuracy: 0.8713 - val_loss: 0.8416 - val_accuracy: 0.8818 - lr: 0.0125 - 44s/epoch - 112ms/step
Epoch 63/250
390/390 - 44s - loss: 0.8670 - accuracy: 0.8736 - val_loss: 0.8634 - val_accuracy: 0.8729 - lr: 0.0125 - 44s/epoch - 112ms/step
Epoch 64/250
390/390 - 44s - loss: 0.8463 - accuracy: 0.8753 - val_loss: 0.7908 - val_accuracy: 0.8900 - lr: 0.0125 - 44s/epoch - 112ms/step
Epoch 65/250
390/390 - 44s - loss: 0.8330 - accuracy: 0.8749 - val_loss: 0.7888 - val_accuracy: 0.8871 - lr: 0.0125 - 44s/epoch - 112ms/step
Epoch 66/250
390/390 - 41s - loss: 0.8207 - accuracy: 0.8755 - val_loss: 0.7431 - val_accuracy: 0.8974 - lr: 0.0125 - 41s/epoch - 105ms/step
Epoch 67/250


Epoch 118/250
390/390 - 43s - loss: 0.4802 - accuracy: 0.9364 - val_loss: 0.5326 - val_accuracy: 0.9208 - lr: 0.0031 - 43s/epoch - 111ms/step
Epoch 119/250
390/390 - 43s - loss: 0.4906 - accuracy: 0.9330 - val_loss: 0.5614 - val_accuracy: 0.9133 - lr: 0.0031 - 43s/epoch - 111ms/step
Epoch 120/250
390/390 - 41s - loss: 0.4844 - accuracy: 0.9340 - val_loss: 0.5611 - val_accuracy: 0.9131 - lr: 0.0031 - 41s/epoch - 104ms/step
Epoch 121/250
390/390 - 44s - loss: 0.4613 - accuracy: 0.9414 - val_loss: 0.5232 - val_accuracy: 0.9255 - lr: 0.0016 - 44s/epoch - 112ms/step
Epoch 122/250
390/390 - 44s - loss: 0.4439 - accuracy: 0.9463 - val_loss: 0.5358 - val_accuracy: 0.9222 - lr: 0.0016 - 44s/epoch - 112ms/step
Epoch 123/250
390/390 - 44s - loss: 0.4431 - accuracy: 0.9466 - val_loss: 0.5357 - val_accuracy: 0.9227 - lr: 0.0016 - 44s/epoch - 112ms/step
Epoch 124/250
390/390 - 40s - loss: 0.4370 - accuracy: 0.9466 - val_loss: 0.5310 - val_accuracy: 0.9220 - lr: 0.0016 - 40s/epoch - 103ms/step
Epoch 

Epoch 175/250
390/390 - 43s - loss: 0.3213 - accuracy: 0.9684 - val_loss: 0.4943 - val_accuracy: 0.9281 - lr: 3.9063e-04 - 43s/epoch - 109ms/step
Epoch 176/250
390/390 - 43s - loss: 0.3221 - accuracy: 0.9673 - val_loss: 0.4853 - val_accuracy: 0.9287 - lr: 3.9063e-04 - 43s/epoch - 109ms/step
Epoch 177/250
390/390 - 40s - loss: 0.3223 - accuracy: 0.9675 - val_loss: 0.4837 - val_accuracy: 0.9297 - lr: 3.9063e-04 - 40s/epoch - 102ms/step
Epoch 178/250
390/390 - 43s - loss: 0.3178 - accuracy: 0.9693 - val_loss: 0.4935 - val_accuracy: 0.9280 - lr: 3.9063e-04 - 43s/epoch - 110ms/step
Epoch 179/250
390/390 - 43s - loss: 0.3227 - accuracy: 0.9669 - val_loss: 0.4756 - val_accuracy: 0.9312 - lr: 3.9063e-04 - 43s/epoch - 111ms/step
Epoch 180/250
390/390 - 40s - loss: 0.3172 - accuracy: 0.9686 - val_loss: 0.4789 - val_accuracy: 0.9306 - lr: 3.9063e-04 - 40s/epoch - 103ms/step
Epoch 181/250
390/390 - 43s - loss: 0.3160 - accuracy: 0.9692 - val_loss: 0.4805 - val_accuracy: 0.9320 - lr: 1.9531e-04 - 4

Epoch 232/250
390/390 - 11s - loss: 0.2942 - accuracy: 0.9739 - val_loss: 0.4751 - val_accuracy: 0.9319 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 233/250
390/390 - 11s - loss: 0.2925 - accuracy: 0.9742 - val_loss: 0.4746 - val_accuracy: 0.9316 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 234/250
390/390 - 11s - loss: 0.2949 - accuracy: 0.9731 - val_loss: 0.4767 - val_accuracy: 0.9302 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 235/250
390/390 - 11s - loss: 0.2938 - accuracy: 0.9731 - val_loss: 0.4743 - val_accuracy: 0.9316 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 236/250
390/390 - 11s - loss: 0.2955 - accuracy: 0.9733 - val_loss: 0.4751 - val_accuracy: 0.9312 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 237/250
390/390 - 11s - loss: 0.2904 - accuracy: 0.9740 - val_loss: 0.4716 - val_accuracy: 0.9323 - lr: 4.8828e-05 - 11s/epoch - 27ms/step
Epoch 238/250
390/390 - 11s - loss: 0.2923 - accuracy: 0.9735 - val_loss: 0.4733 - val_accuracy: 0.9318 - lr: 4.8828e-05 - 11s/epo